In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# How to use\n",
    "Assuming using MacOS or Linux.\n",
    "\n",
    "## 1. Install & Setup\n",
    "Install Python 3.12, then run in a new folder:\n",
    "```bash\n",
    "python3.12 -m venv .venv\n",
    "source .venv/bin/activate\n",
    "python3.12 -m pip install jupyterlab polars altair pyarrow\n",
    "```\n",
    "\n",
    "Save this `.ipynb` file in the same folder you created.\n",
    "\n",
    "## 2. Get Data\n",
    "Generate data on dev using Rancher console, with the command:\n",
    "```bash\n",
    "kubectl get pods --all-namespaces  -o=jsonpath=\"{range .items[*]}{.metadata.namespace}:{.metadata.name}{'\\n'}{range .spec.containers[*]} {.name}:{.resources.requests.cpu}{'\\n'}{end}{'\\n'}{end}\"\n",
    "```\n",
    "\n",
    "Save output to a file in the same folder locally.\n",
    "\n",
    "## 3. Run Jupyter\n",
    "In terminal, run in your local folder:\n",
    "```bash\n",
    "source .venv/bin/activate\n",
    "jupyter lab\n",
    "```\n",
    "\n",
    "Jupyter will now open in your browser, in the Jupyter left sidebar open the `.ipynb` file.\n",
    "\n",
    "## 4. Run the notebook\n",
    "Edit the `input_filename` (2 blocks below) to the name of the file you output in step #2.\n",
    "\n",
    "In the Jupyter menubar at the top, select `Run > Run all Cells` to execute the notebook.\n",
    "\n",
    "Charts will be created below each code cell."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "\"\"\"Imports\"\"\"\n",
    "import re\n",
    "\n",
    "import polars as pl\n",
    "import polars.selectors as cs\n",
    "import altair as alt"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div><style>\n",
       ".dataframe > thead > tr,\n",
       ".dataframe > tbody > tr {\n",
       "  text-align: right;\n",
       "  white-space: pre-wrap;\n",
       "}\n",
       "</style>\n",
       "<small>shape: (5, 1)</small><table border=\"1\" class=\"dataframe\"><thead><tr><th>raw_chunk</th></tr><tr><td>str</td></tr></thead><tbody><tr><td>&quot;acorn:acorn-client-5555d69494-…</td></tr><tr><td>&quot;acorn:acorn-client-fc995884b-h…</td></tr><tr><td>&quot;acorn:acorn-client-fc995884b-m…</td></tr><tr><td>&quot;acorn:acorn-server-55cc8d8d97-…</td></tr><tr><td>&quot;acorn:acorn-server-657f86dfb-4…</td></tr></tbody></table></div>"
      ],
      "text/plain": [
       "shape: (5, 1)\n",
       "┌─────────────────────────────────┐\n",
       "│ raw_chunk                       │\n",
       "│ ---                             │\n",
       "│ str                             │\n",
       "╞═════════════════════════════════╡\n",
       "│ acorn:acorn-client-5555d69494-… │\n",
       "│ acorn:acorn-client-fc995884b-h… │\n",
       "│ acorn:acorn-client-fc995884b-m… │\n",
       "│ acorn:acorn-server-55cc8d8d97-… │\n",
       "│ acorn:acorn-server-657f86dfb-4… │\n",
       "└─────────────────────────────────┘"
      ]
     },
     "execution_count": 2,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "\"\"\"Process the text into a dataframe for cleaning.\"\"\"\n",
    "\n",
    "# replace the file name with your output\n",
    "input_filename = '/Users/sjairam/12-QA-202512.log'\n",
    "with open(input_filename) as raw:\n",
    "    # Just read entire file, it's not that big\n",
    "    raw_text = raw.read()\n",
    "\n",
    "# Split by a blank line, to get pods & containers\n",
    "raw_chunks = re.split(r'\\n\\s*\\n', raw_text)\n",
    "# Convert to dataframe after, can't do regex split easily in Polars\n",
    "raw_df = pl.DataFrame(data={'raw_chunk': raw_chunks})\n",
    "\n",
    "raw_df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div><style>\n",
       ".dataframe > thead > tr,\n",
       ".dataframe > tbody > tr {\n",
       "  text-align: right;\n",
       "  white-space: pre-wrap;\n",
       "}\n",
       "</style>\n",
       "<small>shape: (5, 6)</small><table border=\"1\" class=\"dataframe\"><thead><tr><th>namespace</th><th>pod</th><th>container</th><th>cpu_value</th><th>cpu_units</th><th>milli_cpus</th></tr><tr><td>str</td><td>str</td><td>str</td><td>f64</td><td>str</td><td>f64</td></tr></thead><tbody><tr><td>&quot;acorn&quot;</td><td>&quot;acorn-client-5555d69494-qq8dp&quot;</td><td>&quot;istio-proxy&quot;</td><td>100.0</td><td>&quot;m&quot;</td><td>100.0</td></tr><tr><td>&quot;acorn&quot;</td><td>&quot;acorn-client-5555d69494-qq8dp&quot;</td><td>&quot;acorn-client&quot;</td><td>100.0</td><td>&quot;m&quot;</td><td>100.0</td></tr><tr><td>&quot;acorn&quot;</td><td>&quot;acorn-client-fc995884b-hnzdz&quot;</td><td>&quot;istio-proxy&quot;</td><td>100.0</td><td>&quot;m&quot;</td><td>100.0</td></tr><tr><td>&quot;acorn&quot;</td><td>&quot;acorn-client-fc995884b-hnzdz&quot;</td><td>&quot;acorn-client&quot;</td><td>100.0</td><td>&quot;m&quot;</td><td>100.0</td></tr><tr><td>&quot;acorn&quot;</td><td>&quot;acorn-client-fc995884b-m9fhb&quot;</td><td>&quot;istio-proxy&quot;</td><td>100.0</td><td>&quot;m&quot;</td><td>100.0</td></tr></tbody></table></div>"
      ],
      "text/plain": [
       "shape: (5, 6)\n",
       "┌───────────┬───────────────────────────────┬──────────────┬───────────┬───────────┬────────────┐\n",
       "│ namespace ┆ pod                           ┆ container    ┆ cpu_value ┆ cpu_units ┆ milli_cpus │\n",
       "│ ---       ┆ ---                           ┆ ---          ┆ ---       ┆ ---       ┆ ---        │\n",
       "│ str       ┆ str                           ┆ str          ┆ f64       ┆ str       ┆ f64        │\n",
       "╞═══════════╪═══════════════════════════════╪══════════════╪═══════════╪═══════════╪════════════╡\n",
       "│ acorn     ┆ acorn-client-5555d69494-qq8dp ┆ istio-proxy  ┆ 100.0     ┆ m         ┆ 100.0      │\n",
       "│ acorn     ┆ acorn-client-5555d69494-qq8dp ┆ acorn-client ┆ 100.0     ┆ m         ┆ 100.0      │\n",
       "│ acorn     ┆ acorn-client-fc995884b-hnzdz  ┆ istio-proxy  ┆ 100.0     ┆ m         ┆ 100.0      │\n",
       "│ acorn     ┆ acorn-client-fc995884b-hnzdz  ┆ acorn-client ┆ 100.0     ┆ m         ┆ 100.0      │\n",
       "│ acorn     ┆ acorn-client-fc995884b-m9fhb  ┆ istio-proxy  ┆ 100.0     ┆ m         ┆ 100.0      │\n",
       "└───────────┴───────────────────────────────┴──────────────┴───────────┴───────────┴────────────┘"
      ]
     },
     "execution_count": 3,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "\"\"\"Dataframe cleaning\"\"\"\n",
    "clean_df = raw_df.select(\n",
    "    # Split each row by newline to get list of: [pod, container_1, ...]\n",
    "    split_lists=pl.col(\"raw_chunk\").str.split(\"\\n\"),\n",
    ").select(\n",
    "    # Separate out raw `pod` & `containers` from each inner list\n",
    "    raw_pod=pl.col(\"split_lists\").list.first(),\n",
    "    raw_container=pl.col(\"split_lists\").list.slice(1),\n",
    ").explode(\n",
    "    # Denormalize containers list\n",
    "    \"raw_container\",\n",
    ").select(\n",
    "    # Split out the namespace & pod from `raw_pod`\n",
    "    split_pod=pl.col(\"raw_pod\").str.splitn(\":\", 2).struct.rename_fields([\"namespace\", \"pod\"]),\n",
    "    # Split out container & CPU from `raw_container`\n",
    "    split_container=pl.col(\"raw_container\").str.splitn(\":\", 2).struct.rename_fields([\"container\", \"raw_cpu\"]),\n",
    ").unnest(\n",
    "    # Separate splits into columns\n",
    "    \"split_pod\", \"split_container\",\n",
    ").select(\n",
    "    # Whitespace strip any indented values\n",
    "    cs.string().as_expr().str.strip_chars(), ~cs.string(),\n",
    "    # Extract CPU value vs. unit\n",
    "    cpu_value=pl.col(\"raw_cpu\").str.extract(r\"^([^A-Za-z]+)\").cast(pl.Float64),\n",
    "    cpu_units=pl.col(\"raw_cpu\").str.extract(r\"([A-Za-z]+)$\"),\n",
    ").select(\n",
    "    # Drop any remaining `raw_` columns\n",
    "    ~cs.starts_with(\"raw_\"),\n",
    "    # Convert to milliCPUs\n",
    "    milli_cpus=pl.when(\n",
    "        pl.col(\"cpu_units\") == \"m\"\n",
    "    ).then(\n",
    "        pl.col(\"cpu_value\")\n",
    "    ).when(\n",
    "        pl.col(\"cpu_units\").is_null()\n",
    "    ).then(\n",
    "        pl.col(\"cpu_value\") * 1000.0\n",
    "    ).otherwise(None)\n",
    ")\n",
    "\n",
    "# Materialize final data (ignore null final values)\n",
    "final_df = clean_df.filter(pl.col(\"milli_cpus\").is_not_null())\n",
    "final_df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "\n",
       "<style>\n",
       "  #altair-viz-b0797c58ddaa4d1ca9ede7be70bfdeed.vega-embed {\n",
       "    width: 100%;\n",
       "    display: flex;\n",
       "  }\n",
       "\n",
       "  #altair-viz-b0797c58ddaa4d1ca9ede7be70bfdeed.vega-embed details,\n",
       "  #altair-viz-b0797c58ddaa4d1ca9ede7be70bfdeed.vega-embed details summary {\n",
       "    position: relative;\n",
       "  }\n",
       "</style>\n",
       "<div id=\"altair-viz-b0797c58ddaa4d1ca9ede7be70bfdeed\"></div>\n",
       "<script type=\"text/javascript\">\n",
       "  var VEGA_DEBUG = (typeof VEGA_DEBUG == \"undefined\") ? {} : VEGA_DEBUG;\n",
       "  (function(spec, embedOpt){\n",
       "    let outputDiv = document.currentScript.previousElementSibling;\n",
       "    if (outputDiv.id !== \"altair-viz-b0797c58ddaa4d1ca9ede7be70bfdeed\") {\n",
       "      outputDiv = document.getElementById(\"altair-viz-b0797c58ddaa4d1ca9ede7be70bfdeed\");\n",
       "    }\n",
       "\n",
       "    const paths = {\n",
       "      \"vega\": \"https://cdn.jsdelivr.net/npm/vega@5?noext\",\n",
       "      \"vega-lib\": \"https://cdn.jsdelivr.net/npm/vega-lib?noext\",\n",
       "      \"vega-lite\": \"https://cdn.jsdelivr.net/npm/vega-lite@5.20.1?noext\",\n",
       "      \"vega-embed\": \"https://cdn.jsdelivr.net/npm/vega-embed@6?noext\",\n",
       "    };\n",
       "\n",
       "    function maybeLoadScript(lib, version) {\n",
       "      var key = `${lib.replace(\"-\", \"\")}_version`;\n",
       "      return (VEGA_DEBUG[key] == version) ?\n",
       "        Promise.resolve(paths[lib]) :\n",
       "        new Promise(function(resolve, reject) {\n",
       "          var s = document.createElement('script');\n",
       "          document.getElementsByTagName(\"head\")[0].appendChild(s);\n",
       "          s.async = true;\n",
       "          s.onload = () => {\n",
       "            VEGA_DEBUG[key] = version;\n",
       "            return resolve(paths[lib]);\n",
       "          };\n",
       "          s.onerror = () => reject(`Error loading script: ${paths[lib]}`);\n",
       "          s.src = paths[lib];\n",
       "        });\n",
       "    }\n",
       "\n",
       "    function showError(err) {\n",
       "      outputDiv.innerHTML = `<div class=\"error\" style=\"color:red;\">${err}</div>`;\n",
       "      throw err;\n",
       "    }\n",
       "\n",
       "    function displayChart(vegaEmbed) {\n",
       "      vegaEmbed(outputDiv, spec, embedOpt)\n",
       "        .catch(err => showError(`Javascript Error: ${err.message}<br>This usually means there's a typo in your chart specification. See the javascript console for the full traceback.`));\n",
       "    }\n",
       "\n",
       "    if(typeof define === \"function\" && define.amd) {\n",
       "      requirejs.config({paths});\n",
       "      let deps = [\"vega-embed\"];\n",
       "      require(deps, displayChart, err => showError(`Error loading script: ${err.message}`));\n",
       "    } else {\n",
       "      maybeLoadScript(\"vega\", \"5\")\n",
       "        .then(() => maybeLoadScript(\"vega-lite\", \"5.20.1\"))\n",
       "        .then(() => maybeLoadScript(\"vega-embed\", \"6\"))\n",
       "        .catch(showError)\n",
       "        .then(() => displayChart(vegaEmbed));\n",
       "    }\n",
       "  })({\"config\": {\"view\": {\"continuousWidth\": 300, \"continuousHeight\": 300}}, \"data\": {\"name\": \"data-d271bce9d720ce5e033ca42dbf664587\"}, \"mark\": {\"type\": \"bar\"}, \"encoding\": {\"color\": {\"field\": \"container\", \"type\": \"nominal\"}, \"x\": {\"field\": \"namespace\", \"type\": \"nominal\"}, \"y\": {\"aggregate\": \"sum\", \"field\": \"milli_cpus\", \"type\": \"quantitative\"}}, \"$schema\": \"https://vega.github.io/schema/vega-lite/v5.20.1.json\", \"datasets\": {\"data-d271bce9d720ce5e033ca42dbf664587\": [{\"namespace\": \"acorn\", \"pod\": \"acorn-client-5555d69494-qq8dp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"acorn\", \"pod\": \"acorn-client-5555d69494-qq8dp\", \"container\": \"acorn-client\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"acorn\", \"pod\": \"acorn-client-fc995884b-hnzdz\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"acorn\", \"pod\": \"acorn-client-fc995884b-hnzdz\", \"container\": \"acorn-client\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"acorn\", \"pod\": \"acorn-client-fc995884b-m9fhb\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"acorn\", \"pod\": \"acorn-client-fc995884b-m9fhb\", \"container\": \"acorn-client\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"acorn\", \"pod\": \"acorn-server-55cc8d8d97-x8zdg\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"acorn\", \"pod\": \"acorn-server-55cc8d8d97-x8zdg\", \"container\": \"acorn-server\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"acorn\", \"pod\": \"acorn-server-657f86dfb-4hvlz\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"acorn\", \"pod\": \"acorn-server-657f86dfb-4hvlz\", \"container\": \"acorn-server\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"acorn\", \"pod\": \"acorn-server-657f86dfb-smq2d\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"acorn\", \"pod\": \"acorn-server-657f86dfb-smq2d\", \"container\": \"acorn-server\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"almascripts\", \"pod\": \"request-translator-29415825-6thtp\", \"container\": \"almascripts\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"aqueduct\", \"pod\": \"mystic-9764f6b76-fnngg\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"aqueduct\", \"pod\": \"mystic-9764f6b76-fnngg\", \"container\": \"mystic\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"aqueduct\", \"pod\": \"spicket-667bf88d56-mdqz2\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"aqueduct\", \"pod\": \"winthrop-5d55d959d5-w867l\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"archecker\", \"pod\": \"archecker-567675cf69-x9nvf\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"archecker\", \"pod\": \"archecker-567675cf69-x9nvf\", \"container\": \"archecker\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"archecker\", \"pod\": \"archecker-85dc69cb8-ztbgv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"archecker\", \"pod\": \"archecker-85dc69cb8-ztbgv\", \"container\": \"archecker\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"arclight\", \"pod\": \"arclight-59bb9b68df-ntbxl\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"arclight\", \"pod\": \"arclight-59bb9b68df-ntbxl\", \"container\": \"arclight\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"arclight\", \"pod\": \"arclight-59bb9b68df-ntbxl\", \"container\": \"arclight-proxy\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"arclight\", \"pod\": \"arclight-59bb9b68df-q2smx\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"arclight\", \"pod\": \"arclight-59bb9b68df-q2smx\", \"container\": \"arclight\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"arclight\", \"pod\": \"arclight-59bb9b68df-q2smx\", \"container\": \"arclight-proxy\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"arclight\", \"pod\": \"arclight-aspace-full-n9gkm\", \"container\": \"arclight-aspace-full\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"arclight\", \"pod\": \"arclight-db7cffb64-vqv6h\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"arclight\", \"pod\": \"arclight-db7cffb64-vqv6h\", \"container\": \"arclight\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"arclight\", \"pod\": \"arclight-db7cffb64-vqv6h\", \"container\": \"arclight-proxy\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"arclight\", \"pod\": \"downloads-bucket-5598844d4d-v4pdm\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"arclight\", \"pod\": \"downloads-bucket-5598844d4d-v4pdm\", \"container\": \"downloads-bucket\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"arclight\", \"pod\": \"downloads-bucket-898965d6-n4twq\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"arclight\", \"pod\": \"downloads-bucket-898965d6-n4twq\", \"container\": \"downloads-bucket\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"aspace\", \"pod\": \"aspace-559c8996df-d7dbc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"aspace\", \"pod\": \"aspace-559c8996df-d7dbc\", \"container\": \"aspace\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"aspace\", \"pod\": \"aspace-6f75997556-h2cwq\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"aspace\", \"pod\": \"aspace-6f75997556-h2cwq\", \"container\": \"aspace\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"bibdatalookup\", \"pod\": \"bibdatalookup-6999cfc889-4bmc5\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"bibdatalookup\", \"pod\": \"bibdatalookup-6999cfc889-4bmc5\", \"container\": \"bibdatalookup\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"bibdatalookup\", \"pod\": \"bibdatalookup-c958ffccb-hwtxs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"bibdatalookup\", \"pod\": \"bibdatalookup-c958ffccb-hwtxs\", \"container\": \"bibdatalookup\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"bibdatalookup\", \"pod\": \"bibdatalookup-c958ffccb-thqx5\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"bibdatalookup\", \"pod\": \"bibdatalookup-c958ffccb-thqx5\", \"container\": \"bibdatalookup\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"booklabeler\", \"pod\": \"booklabeler-58f4d6449d-664b8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"booklabeler\", \"pod\": \"booklabeler-58f4d6449d-664b8\", \"container\": \"booklabeler\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"booklabeler\", \"pod\": \"booklabeler-7ccc4cd97b-2k77b\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"booklabeler\", \"pod\": \"booklabeler-7ccc4cd97b-2k77b\", \"container\": \"booklabeler\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"cattle-monitoring-system\", \"pod\": \"alertmanager-rancher-monitoring-alertmanager-0\", \"container\": \"alertmanager\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"cattle-monitoring-system\", \"pod\": \"prometheus-rancher-monitoring-prometheus-0\", \"container\": \"prometheus\", \"cpu_value\": 1250.0, \"cpu_units\": \"m\", \"milli_cpus\": 1250.0}, {\"namespace\": \"cattle-monitoring-system\", \"pod\": \"rancher-monitoring-grafana-5b76cb7f66-f56lg\", \"container\": \"grafana\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"collex\", \"pod\": \"collex-llm-api-deploy-5dd468985f-h7m74\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"collex\", \"pod\": \"collex-llm-api-deploy-5dd468985f-h7m74\", \"container\": \"collex-llm-api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"collex\", \"pod\": \"collex-llm-api-deploy-5fcd74668-5hvtj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"collex\", \"pod\": \"collex-llm-api-deploy-5fcd74668-5hvtj\", \"container\": \"collex-llm-api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"collex\", \"pod\": \"collex-public-frontend-nextjs-deploy-5f5bf6859c-7wmxp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"collex\", \"pod\": \"collex-public-frontend-nextjs-deploy-5f5bf6859c-7wmxp\", \"container\": \"collex-public-frontend-nextjs\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"collex\", \"pod\": \"collex-public-frontend-nextjs-deploy-84698c8676-dk4kc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"collex\", \"pod\": \"collex-public-frontend-nextjs-deploy-84698c8676-dk4kc\", \"container\": \"collex-public-frontend-nextjs\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"collex\", \"pod\": \"collex-search-api-deploy-5996868f8b-d9dvk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"collex\", \"pod\": \"collex-search-api-deploy-5996868f8b-d9dvk\", \"container\": \"collex-search-api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"collex\", \"pod\": \"collex-search-api-deploy-9b5688597-d6xsb\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"collex\", \"pod\": \"collex-search-api-deploy-9b5688597-d6xsb\", \"container\": \"collex-search-api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"curiosity\", \"pod\": \"curiosity-68fdbbd97c-c749x\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"curiosity\", \"pod\": \"curiosity-68fdbbd97c-c749x\", \"container\": \"curiosity\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"curiosity\", \"pod\": \"curiosity-7f6cdf79d8-86tr8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"curiosity\", \"pod\": \"curiosity-7f6cdf79d8-86tr8\", \"container\": \"curiosity\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"curiosity\", \"pod\": \"curiosity-7f6cdf79d8-gr7hj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"curiosity\", \"pod\": \"curiosity-7f6cdf79d8-gr7hj\", \"container\": \"curiosity\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"curiosity\", \"pod\": \"curiosityworker-89f455446-cv854\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"curiosity\", \"pod\": \"curiosityworker-89f455446-cv854\", \"container\": \"curiosityworker\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"dais\", \"pod\": \"dims-6c9b9654cc-f8n64\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"dais\", \"pod\": \"dims-6c9b9654cc-f8n64\", \"container\": \"dims\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"dais\", \"pod\": \"dims-7597788cc-zjn54\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"dais\", \"pod\": \"dims-7597788cc-zjn54\", \"container\": \"dims\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"eda\", \"pod\": \"eda-57bbcd456c-9mhml\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"eda\", \"pod\": \"eda-57bbcd456c-9mhml\", \"container\": \"eda\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"eda\", \"pod\": \"eda-57bbcd456c-9mhml\", \"container\": \"eda-proxy\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"eda\", \"pod\": \"eda-645d6bb5d5-7shhr\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"eda\", \"pod\": \"eda-645d6bb5d5-7shhr\", \"container\": \"eda\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"eda\", \"pod\": \"eda-645d6bb5d5-7shhr\", \"container\": \"eda-proxy\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"eda\", \"pod\": \"eda-iip-668f88f747-9nv4c\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"eda\", \"pod\": \"eda-iip-668f88f747-9nv4c\", \"container\": \"eda-iip\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"eda\", \"pod\": \"eda-iip-f9cfdf4ff-n4qs6\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"eda\", \"pod\": \"eda-iip-f9cfdf4ff-n4qs6\", \"container\": \"eda-iip\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"eda\", \"pod\": \"eda-mc-69b5dfd9b7-7krxw\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"eda\", \"pod\": \"eda-mc-69b5dfd9b7-7krxw\", \"container\": \"eda-mc\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fal-small-images\", \"pod\": \"fal-small-images-5db99dbf4d-7gsvd\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fal-small-images\", \"pod\": \"fal-small-images-5db99dbf4d-7gsvd\", \"container\": \"fal-small-images\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fal-small-images\", \"pod\": \"fal-small-images-6f9fdbc578-x8f2t\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fal-small-images\", \"pod\": \"fal-small-images-6f9fdbc578-x8f2t\", \"container\": \"fal-small-images\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"api-6d688d66b9-2t4rv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"api-6d688d66b9-2t4rv\", \"container\": \"api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"api-85cdbb4745-lc2zc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"api-85cdbb4745-lc2zc\", \"container\": \"api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"api-85cdbb4745-szkq8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"api-85cdbb4745-szkq8\", \"container\": \"api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"loader-5597fc5866-5rwxn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"loader-5597fc5866-5rwxn\", \"container\": \"loader\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"loader-cb8db4956-72n9s\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"loader-cb8db4956-72n9s\", \"container\": \"loader\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"loader-cb8db4956-ffcng\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"loader-cb8db4956-ffcng\", \"container\": \"loader\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"proxy-648fd58784-2lh5p\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"proxy-648fd58784-2lh5p\", \"container\": \"proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"proxy-6d65bb7b4d-lgnv5\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"proxy-6d65bb7b4d-lgnv5\", \"container\": \"proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"ui-5f6bf67c85-mjpg2\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"ui-5f6bf67c85-mjpg2\", \"container\": \"ui\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"ui-646d7b6848-hggrl\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"ui-646d7b6848-hggrl\", \"container\": \"ui\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"ui-646d7b6848-qxkgw\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"fts\", \"pod\": \"ui-646d7b6848-qxkgw\", \"container\": \"ui\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"hgl\", \"pod\": \"gbl-758879d5c9-bphvt\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"hgl\", \"pod\": \"gbl-758879d5c9-bphvt\", \"container\": \"gbl\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"hgl\", \"pod\": \"gbl-944b859d5-fkfqs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"hgl\", \"pod\": \"gbl-944b859d5-fkfqs\", \"container\": \"gbl\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"hgl\", \"pod\": \"gbl-944b859d5-g5mnc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"hgl\", \"pod\": \"gbl-944b859d5-g5mnc\", \"container\": \"gbl\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"hgl\", \"pod\": \"geoserver-6f8b474798-pvmms\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"hgl\", \"pod\": \"geoserver-6f8b474798-pvmms\", \"container\": \"geoserver\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"hgl\", \"pod\": \"geoserver-6f8b474798-pvmms\", \"container\": \"hgl-proxy\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"hgl\", \"pod\": \"geoserver-86b547465d-8q9g9\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"hgl\", \"pod\": \"geoserver-86b547465d-8q9g9\", \"container\": \"geoserver\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"hgl\", \"pod\": \"geoserver-86b547465d-8q9g9\", \"container\": \"hgl-proxy\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"ids\", \"pod\": \"ids-56f564f986-7sbgk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"ids-56f564f986-7sbgk\", \"container\": \"ids\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"ids-68656d5c55-cbjnn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"ids-68656d5c55-cbjnn\", \"container\": \"ids\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"ids-68656d5c55-jss9c\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"ids-68656d5c55-jss9c\", \"container\": \"ids\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"idsmd-6d76f7f9b4-p6dcq\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"idsmd-6d76f7f9b4-p6dcq\", \"container\": \"idsmd\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"idsmd-6d76f7f9b4-qplhk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"idsmd-6d76f7f9b4-qplhk\", \"container\": \"idsmd\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"idsmd-6dfbc9cfbd-9xjc7\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"idsmd-6dfbc9cfbd-9xjc7\", \"container\": \"idsmd\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"iiif-5744dc767f-5ckvv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"iiif-5744dc767f-5ckvv\", \"container\": \"iiif\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"iiif-7b47cf6d9-lqkgv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"iiif-7b47cf6d9-lqkgv\", \"container\": \"iiif\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"iiif-7b47cf6d9-m5jjh\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"iiif-7b47cf6d9-m5jjh\", \"container\": \"iiif\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"iiif-updater-7557ccccff-bwgcf\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"iiif-updater-7557ccccff-bwgcf\", \"container\": \"iiif-updater\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"pds-f58f57594-8cwls\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"pds-f58f57594-8cwls\", \"container\": \"pds\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"pds-f58f57594-ks495\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"ids\", \"pod\": \"pds-f58f57594-ks495\", \"container\": \"pds\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"imgconv\", \"pod\": \"imgconv-84cb96d846-88cpx\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"imgconv\", \"pod\": \"imgconv-84cb96d846-88cpx\", \"container\": \"imgconv\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 125.0}, {\"namespace\": \"imgconv\", \"pod\": \"imgconv-84cb96d846-zg2mk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"imgconv\", \"pod\": \"imgconv-84cb96d846-zg2mk\", \"container\": \"imgconv\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 125.0}, {\"namespace\": \"istio-system\", \"pod\": \"istio-apiprivate-ingressgateway-7c4bd499d9-7lctf\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"istio-system\", \"pod\": \"istio-ingressgateway-57c7fdd696-rdx95\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"istio-system\", \"pod\": \"istio-private-ingressgateway-7b69c99dc-sh8h8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"istio-system\", \"pod\": \"istio-server-ingressgateway-66dd9898b9-6rngs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"istio-system\", \"pod\": \"istiod-84cd86b54d-vrkvc\", \"container\": \"discovery\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"istio-system\", \"pod\": \"jaeger-7799f58d6-7h4n2\", \"container\": \"jaegertracing\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"istio-system\", \"pod\": \"kiali-59cc5b5f4d-jsf4f\", \"container\": \"kiali\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"jobmonitor\", \"pod\": \"jobmonitor-d597f6ddf-7fqkc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jobmonitor\", \"pod\": \"jobmonitor-d597f6ddf-7fqkc\", \"container\": \"jobmonitor\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"jobmonitor\", \"pod\": \"jobmonitor-d597f6ddf-cfsrj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jobmonitor\", \"pod\": \"jobmonitor-d597f6ddf-cfsrj\", \"container\": \"jobmonitor\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"jstor\", \"pod\": \"downloads-bucket-ead-7cb584c88b-r669w\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"downloads-bucket-ead-7cb584c88b-r669w\", \"container\": \"downloads-bucket-ead\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"jstor\", \"pod\": \"downloads-bucket-via-594bdb8fbd-44br7\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"downloads-bucket-via-594bdb8fbd-44br7\", \"container\": \"downloads-bucket-via\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-aggregator-6cc4f4ff5b-jhw9h\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-aggregator-6cc4f4ff5b-jhw9h\", \"container\": \"jstor-aggregator\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-aggregator-6cc4f4ff5b-vd4n4\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-aggregator-6cc4f4ff5b-vd4n4\", \"container\": \"jstor-aggregator\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-aggregator-retro-56b74fdc4-hvbz8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-aggregator-retro-56b74fdc4-hvbz8\", \"container\": \"jstor-aggregator-retro\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-harvester-759467f6f6-h247h\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-harvester-759467f6f6-h247h\", \"container\": \"jstor-harvester\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-harvester-759467f6f6-trpdx\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-harvester-759467f6f6-trpdx\", \"container\": \"jstor-harvester\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-harvester-retro-f9d86ffbf-5hq6x\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-harvester-retro-f9d86ffbf-5hq6x\", \"container\": \"jstor-harvester-retro\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-integration-55ccf5795d-7tn5j\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-integration-55ccf5795d-7tn5j\", \"container\": \"jstor-integration\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-integration-55ccf5795d-fgblc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-integration-55ccf5795d-fgblc\", \"container\": \"jstor-integration\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-publisher-79996b59f-hltwz\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-publisher-79996b59f-hltwz\", \"container\": \"jstor-publisher\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-publisher-79996b59f-j5dqd\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-publisher-79996b59f-j5dqd\", \"container\": \"jstor-publisher\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-publisher-retro-76d9bb94d-2x2k8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-publisher-retro-76d9bb94d-2x2k8\", \"container\": \"jstor-publisher-retro\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-transformer-5ff7b76798-4c2fp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-transformer-5ff7b76798-4c2fp\", \"container\": \"jstor-transformer\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-transformer-5ff7b76798-qgjwc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-transformer-5ff7b76798-qgjwc\", \"container\": \"jstor-transformer\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-transformer-retro-56d449ff99-tcp2r\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"jstor\", \"pod\": \"jstor-transformer-retro-56d449ff99-tcp2r\", \"container\": \"jstor-transformer-retro\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 500.0}, {\"namespace\": \"kafka\", \"pod\": \"lts-kafka-dual-role-0\", \"container\": \"kafka\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"kafka\", \"pod\": \"lts-kafka-dual-role-1\", \"container\": \"kafka\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"kafka\", \"pod\": \"lts-kafka-dual-role-2\", \"container\": \"kafka\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"kafka\", \"pod\": \"lts-kafka-entity-operator-c464ccbcc-8xd4k\", \"container\": \"topic-operator\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"kafka\", \"pod\": \"lts-kafka-entity-operator-c464ccbcc-8xd4k\", \"container\": \"user-operator\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"kafka\", \"pod\": \"strimzi-cluster-operator-59bb568c45-vcg8h\", \"container\": \"strimzi-cluster-operator\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 200.0}, {\"namespace\": \"kube-system\", \"pod\": \"cloud-controller-manager-ip-10-137-166-22\", \"container\": \"cloud-controller-manager\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"cloud-controller-manager-ip-10-137-166-26\", \"container\": \"cloud-controller-manager\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"cloud-controller-manager-ip-10-137-166-27\", \"container\": \"cloud-controller-manager\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-controller-69dc5b4c8c-4cxt5\", \"container\": \"csi-provisioner\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-controller-69dc5b4c8c-4cxt5\", \"container\": \"csi-resizer\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-controller-69dc5b4c8c-4cxt5\", \"container\": \"csi-snapshotter\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-controller-69dc5b4c8c-4cxt5\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-controller-69dc5b4c8c-4cxt5\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-7ttzk\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-7ttzk\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-7ttzk\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-8ntsn\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-8ntsn\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-8ntsn\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-9rxl4\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-9rxl4\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-9rxl4\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-cvmrw\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-cvmrw\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-cvmrw\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-hcrps\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-hcrps\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-hcrps\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-khqwx\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-khqwx\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-khqwx\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-m8fws\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-m8fws\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-m8fws\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-wcm6w\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-wcm6w\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-wcm6w\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-wdpbm\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-wdpbm\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-wdpbm\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-q7bpm\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-q7bpm\", \"container\": \"csi-provisioner\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-q7bpm\", \"container\": \"csi-attacher\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-q7bpm\", \"container\": \"csi-snapshotter\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-q7bpm\", \"container\": \"csi-resizer\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-q7bpm\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-wstjf\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-wstjf\", \"container\": \"csi-provisioner\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-wstjf\", \"container\": \"csi-attacher\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-wstjf\", \"container\": \"csi-snapshotter\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-wstjf\", \"container\": \"csi-resizer\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-wstjf\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-9wptm\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-9wptm\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-9wptm\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-9zb49\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-9zb49\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-9zb49\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-dtmqn\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-dtmqn\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-dtmqn\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-gk7wz\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-gk7wz\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-gk7wz\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-j58cz\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-j58cz\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-j58cz\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-jrb29\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-jrb29\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-jrb29\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-m8tlx\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-m8tlx\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-m8tlx\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-v9wgj\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-v9wgj\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-v9wgj\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-zl2bj\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-zl2bj\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-zl2bj\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"kube-system\", \"pod\": \"etcd-ip-10-137-166-22\", \"container\": \"etcd\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 200.0}, {\"namespace\": \"kube-system\", \"pod\": \"etcd-ip-10-137-166-26\", \"container\": \"etcd\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 200.0}, {\"namespace\": \"kube-system\", \"pod\": \"etcd-ip-10-137-166-27\", \"container\": \"etcd\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 200.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-apiserver-ip-10-137-166-22\", \"container\": \"kube-apiserver\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-apiserver-ip-10-137-166-26\", \"container\": \"kube-apiserver\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-apiserver-ip-10-137-166-27\", \"container\": \"kube-apiserver\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-controller-manager-ip-10-137-166-22\", \"container\": \"kube-controller-manager\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 200.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-controller-manager-ip-10-137-166-26\", \"container\": \"kube-controller-manager\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 200.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-controller-manager-ip-10-137-166-27\", \"container\": \"kube-controller-manager\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 200.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-166-22\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-166-26\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-166-27\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-167-20\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-167-22\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-167-24\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-167-28\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-167-29\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-167-30\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-scheduler-ip-10-137-166-22\", \"container\": \"kube-scheduler\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-scheduler-ip-10-137-166-26\", \"container\": \"kube-scheduler\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"kube-scheduler-ip-10-137-166-27\", \"container\": \"kube-scheduler\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-582tl\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-5hxqk\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-7k9m5\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-gqpxz\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-lxjqn\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-r4slr\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-srzln\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-tqd47\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-v8nt6\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-coredns-rke2-coredns-85d6696775-ph7v7\", \"container\": \"coredns\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-coredns-rke2-coredns-85d6696775-q9clm\", \"container\": \"coredns\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-coredns-rke2-coredns-autoscaler-665b7f6f86-rtrl7\", \"container\": \"autoscaler\", \"cpu_value\": 25.0, \"cpu_units\": \"m\", \"milli_cpus\": 25.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-5d7xf\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-9dzcz\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-d7qt6\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-hh6hw\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-hxx5p\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-k5j7n\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-nkh47\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-sjqbh\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-v6sl6\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-metrics-server-7c4c577547-9zj5g\", \"container\": \"metrics-server\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-collections-547756785c-pnt49\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-collections-547756785c-pnt49\", \"container\": \"lc-collections\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-collections-547756785c-vld6j\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-collections-547756785c-vld6j\", \"container\": \"lc-collections\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-collectionsbuilder-7c8bd8575-mz8hx\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-collectionsbuilder-7c8bd8575-mz8hx\", \"container\": \"lc-collectionsbuilder\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-control-server-75b8bfc566-pckkn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-control-server-75b8bfc566-pckkn\", \"container\": \"lc-control-server\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-csv-df57cbc58-gg4w9\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-csv-df57cbc58-gg4w9\", \"container\": \"lc-csv\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-csv-df57cbc58-z6wgv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-csv-df57cbc58-z6wgv\", \"container\": \"lc-csv\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-ingest-868b55594f-2gkjn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-ingest-868b55594f-2gkjn\", \"container\": \"lc-ingest\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-ingest-868b55594f-r6tqf\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-ingest-868b55594f-r6tqf\", \"container\": \"lc-ingest\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-oai-69887f59b5-rk5hs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-oai-69887f59b5-rk5hs\", \"container\": \"lc-oai\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-oai-69887f59b5-vlclq\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-oai-69887f59b5-vlclq\", \"container\": \"lc-oai\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"librarycloud-694f6dfdbb-r8htx\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"librarycloud\", \"pod\": \"librarycloud-694f6dfdbb-r8htx\", \"container\": \"librarycloud\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"listview\", \"pod\": \"listview-7bc65f4c45-4wxwh\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"listview\", \"pod\": \"listview-7bc65f4c45-4wxwh\", \"container\": \"listview\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"listview\", \"pod\": \"listview-7bc65f4c45-kxk44\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"listview\", \"pod\": \"listview-7bc65f4c45-kxk44\", \"container\": \"listview\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"listview\", \"pod\": \"listview-md-65f9569858-cwv8f\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"listview\", \"pod\": \"listview-md-65f9569858-cwv8f\", \"container\": \"listview-md\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"loki\", \"pod\": \"loki-0\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"loki\", \"pod\": \"loki-grafana-8cd994887-flltj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-762cs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-98b89\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-b84px\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-blcb5\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-mgbjp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-mnf7j\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-nhz77\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-pq6b5\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-z2hjv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"lts-pipelines\", \"pod\": \"collex-main-ingest-pipeline-finalize-mongo-job-acqpkr6m\", \"container\": \"base\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-pgbouncer-deploy-dbdd6cdf7-8zdj8\", \"container\": \"pgbouncer\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-scheduler-deploy-6c9f65c84c-f6wb6\", \"container\": \"scheduler\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-scheduler-deploy-6c9f65c84c-qmddr\", \"container\": \"scheduler\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-triggerer-deploy-799c4ddc47-kcshl\", \"container\": \"triggerer\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-webserver-deploy-cb459dc58-5bjpv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-webserver-deploy-cb459dc58-5bjpv\", \"container\": \"webserver\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-webserver-deploy-cb459dc58-x4zfp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-webserver-deploy-cb459dc58-x4zfp\", \"container\": \"webserver\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mds\", \"pod\": \"mdsrv-6b9b5568b4-nnkvl\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mds\", \"pod\": \"mdsrv-6b9b5568b4-nnkvl\", \"container\": \"mdsrv\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 125.0}, {\"namespace\": \"mds\", \"pod\": \"mdsrv-6b9b5568b4-v4pkz\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mds\", \"pod\": \"mdsrv-6b9b5568b4-v4pkz\", \"container\": \"mdsrv\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 125.0}, {\"namespace\": \"mds\", \"pod\": \"mqsrv-7f44bc5b4f-dqn9h\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mds\", \"pod\": \"mqsrv-7f44bc5b4f-dqn9h\", \"container\": \"mqsrv\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 125.0}, {\"namespace\": \"mds\", \"pod\": \"mqsrv-7f44bc5b4f-v8wtk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mds\", \"pod\": \"mqsrv-7f44bc5b4f-v8wtk\", \"container\": \"mqsrv\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 125.0}, {\"namespace\": \"mds\", \"pod\": \"ocfl-storage-ro-5fb6594c45-8jgzn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mds\", \"pod\": \"ocfl-storage-ro-5fb6594c45-8jgzn\", \"container\": \"ocfl-storage-ro\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 125.0}, {\"namespace\": \"mds\", \"pod\": \"s3pather-675d9bbcdb-46znp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mds\", \"pod\": \"s3pather-675d9bbcdb-46znp\", \"container\": \"s3pather\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 125.0}, {\"namespace\": \"mds\", \"pod\": \"s3pather-675d9bbcdb-crcj5\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mds\", \"pod\": \"s3pather-675d9bbcdb-crcj5\", \"container\": \"s3pather\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 125.0}, {\"namespace\": \"mds\", \"pod\": \"solr2mds-64f8bbb966-8fvsc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mds\", \"pod\": \"solr2mds-64f8bbb966-8fvsc\", \"container\": \"solr2mds\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 125.0}, {\"namespace\": \"mds\", \"pod\": \"solr2mds-64f8bbb966-hfjtk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mds\", \"pod\": \"solr2mds-64f8bbb966-hfjtk\", \"container\": \"solr2mds\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 125.0}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-drs-gateway-b595f977c-sj4jt\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-drs-gateway-b595f977c-sj4jt\", \"container\": \"md-operational-drs-gateway\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-mongo-file-ingestor-6cff445458-q5rvt\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-mongo-file-ingestor-6cff445458-q5rvt\", \"container\": \"md-operational-mongo-file-ingestor\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-mongo-ingestor-76fb97fc65-d6bvt\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-mongo-ingestor-76fb97fc65-d6bvt\", \"container\": \"md-operational-mongo-ingestor\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-solr-ingestor-566dcfbbb-6dqt9\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-solr-ingestor-566dcfbbb-6dqt9\", \"container\": \"md-operational-solr-ingestor\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-embed-655d9bff6d-d6rqn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-embed-655d9bff6d-d6rqn\", \"container\": \"mps-embed\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-embed-655d9bff6d-tdnpw\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-embed-655d9bff6d-tdnpw\", \"container\": \"mps-embed\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-viewer-57594b8f58-8b4tt\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-viewer-57594b8f58-8b4tt\", \"container\": \"mps-viewer\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-viewer-57594b8f58-hb8k8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-viewer-57594b8f58-hb8k8\", \"container\": \"mps-viewer\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"mps\", \"pod\": \"access-service-c7fd6b5cf-6zvn6\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"access-service-c7fd6b5cf-6zvn6\", \"container\": \"access-service\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"asset-cache-mgr-577fcd89fd-74txv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"asset-cache-mgr-577fcd89fd-74txv\", \"container\": \"asset-cache-mgr\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"asset-db-connector-c55b55685-2drcw\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"asset-db-connector-c55b55685-2drcw\", \"container\": \"asset-db-connector\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"asset-delivery-files-7f7b94777b-ccf4p\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"asset-delivery-files-7f7b94777b-ccf4p\", \"container\": \"asset-delivery-files\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"asset-delivery-images-8589bf4769-fvm2g\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"asset-delivery-images-8589bf4769-fvm2g\", \"container\": \"asset-delivery-images\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"asset-delivery-thumbnails-b98548df-rnp2c\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"asset-delivery-thumbnails-b98548df-rnp2c\", \"container\": \"asset-delivery-thumbnails\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"authorization-6ffbfdf8ff-t9npw\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"authorization-6ffbfdf8ff-t9npw\", \"container\": \"authorization\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"cache-populator-85b8b5fb9d-bvtbd\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"cache-populator-85b8b5fb9d-bvtbd\", \"container\": \"cache-populator\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-aws-kak-d4d77787b-hqb7p\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-aws-kak-d4d77787b-hqb7p\", \"container\": \"cantaloupe-aws-kak\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-aws-ojpg-6ff6fc5d67-gh9bs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-aws-ojpg-6ff6fc5d67-gh9bs\", \"container\": \"cantaloupe-aws-ojpg\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-proxy-b894d47f6-s4pjt\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-proxy-b894d47f6-s4pjt\", \"container\": \"cantaloupe-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-was-kak-74f9979b6f-7ffz8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-was-kak-74f9979b6f-7ffz8\", \"container\": \"cantaloupe-was-kak\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-was-ojpg-bcdcbff99-pjjvf\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-was-ojpg-bcdcbff99-pjjvf\", \"container\": \"cantaloupe-was-ojpg\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"citation-api-db84f7486-w8t5v\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"citation-api-db84f7486-w8t5v\", \"container\": \"citation-api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"deliverys3cache-979f7658-8xkjs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"deliverys3cache-979f7658-8xkjs\", \"container\": \"deliverys3cache\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"deliverys3proxy-56b9db6c4-lvhsn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"deliverys3proxy-56b9db6c4-lvhsn\", \"container\": \"deliverys3proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"ids-redirects-564c7d86c8-94mkk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"ids-redirects-564c7d86c8-94mkk\", \"container\": \"ids-redirects\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"iiif-converter-559d794756-sg9x4\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"iiif-converter-559d794756-sg9x4\", \"container\": \"iiif-converter\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"iiif-presentation-688b56566-fj86j\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"iiif-presentation-688b56566-fj86j\", \"container\": \"iiif-presentation\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"iiif-synchronizer-updater-6cdd57d955-v7798\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"iiif-synchronizer-updater-6cdd57d955-v7798\", \"container\": \"iiif-synchronizer-updater\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"inlib-5c8bb7558f-g9gr6\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"inlib-5c8bb7558f-g9gr6\", \"container\": \"inlib\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"login-8558f4c44f-l2rb4\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"login-8558f4c44f-l2rb4\", \"container\": \"login\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"lookup-58796fdc47-x2mzf\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"lookup-58796fdc47-x2mzf\", \"container\": \"lookup\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"ocrsearch-58876c7b69-55q2r\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"ocrsearch-58876c7b69-55q2r\", \"container\": \"ocrsearch\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"pdf-download-6756c5567c-lw7vq\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"pdf-download-6756c5567c-lw7vq\", \"container\": \"pdf-download\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"url-redirects-86666f67b7-qhg79\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mps\", \"pod\": \"url-redirects-86666f67b7-qhg79\", \"container\": \"url-redirects\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-cache-mgr-67c44b5c54-gltfs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-cache-mgr-67c44b5c54-gltfs\", \"container\": \"asset-cache-mgr\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-copy-7f8d5b456b-h6q22\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-copy-7f8d5b456b-h6q22\", \"container\": \"asset-copy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-db-connector-54969d4596-tzppr\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-db-connector-54969d4596-tzppr\", \"container\": \"asset-db-connector\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-validation-85488cf799-jkb4v\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-validation-85488cf799-jkb4v\", \"container\": \"asset-validation\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"authorization-5d9fd8cd78-lrs77\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"authorization-5d9fd8cd78-lrs77\", \"container\": \"authorization\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"deactivator-7cf7d8cc5-9v6w9\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"deactivator-7cf7d8cc5-9v6w9\", \"container\": \"deactivator\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-converter-6496d46b75-4rkd9\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-converter-6496d46b75-4rkd9\", \"container\": \"iiif-converter\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-metadata-updater-5c46b6877d-79pvn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-metadata-updater-5c46b6877d-79pvn\", \"container\": \"iiif-metadata-updater\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-services-cdc49965d-8l2l5\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-services-cdc49965d-8l2l5\", \"container\": \"iiif-services\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-synchronizer-iterator-64cf46bb55-thns9\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-synchronizer-iterator-64cf46bb55-thns9\", \"container\": \"iiif-synchronizer-iterator\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-synchronizer-updater-5f555df75d-qx7lj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-synchronizer-updater-5f555df75d-qx7lj\", \"container\": \"iiif-synchronizer-updater\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-task-manager-6b598c4fcd-nrrcd\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-task-manager-6b598c4fcd-nrrcd\", \"container\": \"iiif-task-manager\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-validator-f77d96fdc-jn427\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-validator-f77d96fdc-jn427\", \"container\": \"iiif-validator\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"ingest-6c656dff57-8lv8c\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"ingest-6c656dff57-8lv8c\", \"container\": \"ingest\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"ltsfileservice-776768896d-6wjcd\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"ltsfileservice-776768896d-6wjcd\", \"container\": \"ltsfileservice\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"mpscron-29415859-562kn\", \"container\": \"mpscron\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"mpscron-29415860-pq8fq\", \"container\": \"mpscron\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"multi-asset-ingest-worker-f9fc86888-dfwjb\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"multi-asset-ingest-worker-f9fc86888-dfwjb\", \"container\": \"multi-asset-ingest-worker\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"queue-monitor-5897c74948-skctx\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"mpsadm\", \"pod\": \"queue-monitor-5897c74948-skctx\", \"container\": \"queue-monitor\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"nrs\", \"pod\": \"resolver-7d98f98bb5-7hpwv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"nrs\", \"pod\": \"resolver-7d98f98bb5-7hpwv\", \"container\": \"resolver\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"nrs\", \"pod\": \"resolver-cache-5bf6877c57-24566\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"nrs\", \"pod\": \"resolver-cache-5bf6877c57-24566\", \"container\": \"resolver-cache\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"nrsadm\", \"pod\": \"cache-refresh-29407860-5fhwr\", \"container\": \"cache-refresh\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"nrsadm\", \"pod\": \"cachemgr-7dc594c44f-2cqzk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"nrsadm\", \"pod\": \"cachemgr-7dc594c44f-2cqzk\", \"container\": \"cachemgr\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"nrsadm\", \"pod\": \"drs-file-urn-29415720-97npj\", \"container\": \"drs-file-urn\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"nrsadm\", \"pod\": \"drs-file-urn-29415780-4l5xw\", \"container\": \"drs-file-urn\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"nrsadm\", \"pod\": \"drs-file-urn-29415840-bkbtz\", \"container\": \"drs-file-urn\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"nrsadm\", \"pod\": \"nrs-admin-api-796f6c64f6-b4vtn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"nrsadm\", \"pod\": \"nrs-admin-api-796f6c64f6-b4vtn\", \"container\": \"nrs-admin-api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"nrsadm\", \"pod\": \"nrs-admin-ui-api-79b69dd46-zgn86\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"nrsadm\", \"pod\": \"nrs-admin-ui-api-79b69dd46-zgn86\", \"container\": \"nrs-admin-ui-api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"nrsadm\", \"pod\": \"nrs-admin-ui-client-5fc468ff74-jwkjb\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"nrsadm\", \"pod\": \"nrs-admin-ui-client-5fc468ff74-jwkjb\", \"container\": \"nrs-admin-ui-client\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 10.0}, {\"namespace\": \"olivia\", \"pod\": \"olivia-6998686d85-hkfcj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"olivia\", \"pod\": \"olivia-6998686d85-hkfcj\", \"container\": \"olivia\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"olivia\", \"pod\": \"olivia-md-768f8b66f-2k8zj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"olivia\", \"pod\": \"olivia-md-768f8b66f-2k8zj\", \"container\": \"olivia-md\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"policy\", \"pod\": \"policy-admin-6ccb9bbbd6-6gvnt\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"policy\", \"pod\": \"policy-admin-6ccb9bbbd6-6gvnt\", \"container\": \"policy-admin\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"policy\", \"pod\": \"policy-admin-6ccb9bbbd6-nm2kj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"policy\", \"pod\": \"policy-admin-6ccb9bbbd6-nm2kj\", \"container\": \"policy-admin\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"delivery-file-848848548b-kw9qh\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"delivery-file-848848548b-kw9qh\", \"container\": \"delivery-file\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notification-dispatcher-f785b5b66-6tjtg\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notification-dispatcher-f785b5b66-6tjtg\", \"container\": \"drs-notification-dispatcher\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notification-worker-fd5f9b59-2pndn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notification-worker-fd5f9b59-2pndn\", \"container\": \"drs-notification-worker\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notifier-7988b89fd5-pdp8w\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notifier-7988b89fd5-pdp8w\", \"container\": \"drs-notifier\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notifier-listener-86bc4b4bf5-9nvnb\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notifier-listener-86bc4b4bf5-9nvnb\", \"container\": \"drs-notifier-listener\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"lts-pipeline-external-queue-77ff4fc975-6kxzc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"lts-pipeline-external-queue-77ff4fc975-6kxzc\", \"container\": \"lts-pipeline-external-queue\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"mps-stager-57945556f6-2cfxs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"mps-stager-57945556f6-2cfxs\", \"container\": \"mps-stager\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"nrs-update-755766d4f6-zjl5c\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"nrs-update-755766d4f6-zjl5c\", \"container\": \"nrs-update\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"sds-delivery-ingest-845cdc786f-6dbck\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"post-drs-msg\", \"pod\": \"sds-delivery-ingest-845cdc786f-6dbck\", \"container\": \"sds-delivery-ingest\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"presto\", \"pod\": \"presto-5c6b88f4c8-qgv49\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"presto\", \"pod\": \"presto-5c6b88f4c8-qgv49\", \"container\": \"presto\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"presto\", \"pod\": \"presto-5c6b88f4c8-t64vp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"presto\", \"pod\": \"presto-5c6b88f4c8-t64vp\", \"container\": \"presto\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"sds\", \"pod\": \"metadata-7779554449-d5klv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"sds\", \"pod\": \"metadata-7779554449-d5klv\", \"container\": \"metadata\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"sds\", \"pod\": \"sds-85db5db78f-74lkd\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"sds\", \"pod\": \"sds-85db5db78f-74lkd\", \"container\": \"sds\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"sds\", \"pod\": \"wowza-c4ffc4c6d-9zgt9\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"sds\", \"pod\": \"wowza-c4ffc4c6d-9zgt9\", \"container\": \"wowza\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"secureexit\", \"pod\": \"secureexit-5c5959667b-lrrm7\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"secureexit\", \"pod\": \"secureexit-5c5959667b-lrrm7\", \"container\": \"secureexit\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1000.0}, {\"namespace\": \"shared-shelf-linker\", \"pod\": \"sharedshelf-7c88d954ff-2jxqp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"shared-shelf-linker\", \"pod\": \"sharedshelf-7c88d954ff-2jxqp\", \"container\": \"sharedshelf\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"solr\", \"pod\": \"solr-qa-solrcloud-zookeeper-0\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"solr\", \"pod\": \"solr-qa-solrcloud-zookeeper-0\", \"container\": \"zookeeper\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"solr\", \"pod\": \"solr-qa-solrcloud-zookeeper-1\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"solr\", \"pod\": \"solr-qa-solrcloud-zookeeper-1\", \"container\": \"zookeeper\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"solr\", \"pod\": \"solr-qa-solrcloud-zookeeper-2\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 100.0}, {\"namespace\": \"solr\", \"pod\": \"solr-qa-solrcloud-zookeeper-2\", \"container\": \"zookeeper\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 250.0}, {\"namespace\": \"testkube\", \"pod\": \"testkube-api-server-86b89d59fc-sr2jx\", \"container\": \"testkube-api\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 200.0}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-api-5d6c7db85b-dhlm2\", \"container\": \"testkube-cloud-api\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 50.0}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-dex-665b57765c-khls9\", \"container\": \"dex\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 50.0}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-nats-0\", \"container\": \"nats\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 50.0}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-nats-0\", \"container\": \"reloader\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 50.0}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-nats-1\", \"container\": \"nats\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 50.0}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-nats-1\", \"container\": \"reloader\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 50.0}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-nats-2\", \"container\": \"nats\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 50.0}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-nats-2\", \"container\": \"reloader\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 50.0}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-ui-7f868dfff4-mh2c9\", \"container\": \"testkube-cloud-ui\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 50.0}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-worker-service-78c654bddb-crjgn\", \"container\": \"testkube-worker-service\", \"cpu_value\": 75.0, \"cpu_units\": \"m\", \"milli_cpus\": 75.0}]}}, {\"mode\": \"vega-lite\"});\n",
       "</script>"
      ],
      "text/plain": [
       "alt.Chart(...)"
      ]
     },
     "execution_count": 4,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "\"\"\"Plot data\"\"\"\n",
    "alt.Chart(final_df).mark_bar().encode( # type: ignore\n",
    "    x='namespace',\n",
    "    y='sum(milli_cpus)',\n",
    "    color='container',\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "\n",
       "<style>\n",
       "  #altair-viz-8318fcbf1c044c7fb2f7c5eb4bd93814.vega-embed {\n",
       "    width: 100%;\n",
       "    display: flex;\n",
       "  }\n",
       "\n",
       "  #altair-viz-8318fcbf1c044c7fb2f7c5eb4bd93814.vega-embed details,\n",
       "  #altair-viz-8318fcbf1c044c7fb2f7c5eb4bd93814.vega-embed details summary {\n",
       "    position: relative;\n",
       "  }\n",
       "</style>\n",
       "<div id=\"altair-viz-8318fcbf1c044c7fb2f7c5eb4bd93814\"></div>\n",
       "<script type=\"text/javascript\">\n",
       "  var VEGA_DEBUG = (typeof VEGA_DEBUG == \"undefined\") ? {} : VEGA_DEBUG;\n",
       "  (function(spec, embedOpt){\n",
       "    let outputDiv = document.currentScript.previousElementSibling;\n",
       "    if (outputDiv.id !== \"altair-viz-8318fcbf1c044c7fb2f7c5eb4bd93814\") {\n",
       "      outputDiv = document.getElementById(\"altair-viz-8318fcbf1c044c7fb2f7c5eb4bd93814\");\n",
       "    }\n",
       "\n",
       "    const paths = {\n",
       "      \"vega\": \"https://cdn.jsdelivr.net/npm/vega@5?noext\",\n",
       "      \"vega-lib\": \"https://cdn.jsdelivr.net/npm/vega-lib?noext\",\n",
       "      \"vega-lite\": \"https://cdn.jsdelivr.net/npm/vega-lite@5.20.1?noext\",\n",
       "      \"vega-embed\": \"https://cdn.jsdelivr.net/npm/vega-embed@6?noext\",\n",
       "    };\n",
       "\n",
       "    function maybeLoadScript(lib, version) {\n",
       "      var key = `${lib.replace(\"-\", \"\")}_version`;\n",
       "      return (VEGA_DEBUG[key] == version) ?\n",
       "        Promise.resolve(paths[lib]) :\n",
       "        new Promise(function(resolve, reject) {\n",
       "          var s = document.createElement('script');\n",
       "          document.getElementsByTagName(\"head\")[0].appendChild(s);\n",
       "          s.async = true;\n",
       "          s.onload = () => {\n",
       "            VEGA_DEBUG[key] = version;\n",
       "            return resolve(paths[lib]);\n",
       "          };\n",
       "          s.onerror = () => reject(`Error loading script: ${paths[lib]}`);\n",
       "          s.src = paths[lib];\n",
       "        });\n",
       "    }\n",
       "\n",
       "    function showError(err) {\n",
       "      outputDiv.innerHTML = `<div class=\"error\" style=\"color:red;\">${err}</div>`;\n",
       "      throw err;\n",
       "    }\n",
       "\n",
       "    function displayChart(vegaEmbed) {\n",
       "      vegaEmbed(outputDiv, spec, embedOpt)\n",
       "        .catch(err => showError(`Javascript Error: ${err.message}<br>This usually means there's a typo in your chart specification. See the javascript console for the full traceback.`));\n",
       "    }\n",
       "\n",
       "    if(typeof define === \"function\" && define.amd) {\n",
       "      requirejs.config({paths});\n",
       "      let deps = [\"vega-embed\"];\n",
       "      require(deps, displayChart, err => showError(`Error loading script: ${err.message}`));\n",
       "    } else {\n",
       "      maybeLoadScript(\"vega\", \"5\")\n",
       "        .then(() => maybeLoadScript(\"vega-lite\", \"5.20.1\"))\n",
       "        .then(() => maybeLoadScript(\"vega-embed\", \"6\"))\n",
       "        .catch(showError)\n",
       "        .then(() => displayChart(vegaEmbed));\n",
       "    }\n",
       "  })({\"config\": {\"view\": {\"continuousWidth\": 300, \"continuousHeight\": 300}}, \"data\": {\"name\": \"data-4f53cda18c2baa0c0354bb5f9a3ecbe5\"}, \"mark\": {\"type\": \"bar\"}, \"encoding\": {\"color\": {\"field\": \"container\", \"type\": \"nominal\"}, \"x\": {\"field\": \"pod\", \"type\": \"nominal\"}, \"y\": {\"aggregate\": \"sum\", \"field\": \"milli_cpus\", \"type\": \"quantitative\"}}, \"$schema\": \"https://vega.github.io/schema/vega-lite/v5.20.1.json\", \"datasets\": {\"data-4f53cda18c2baa0c0354bb5f9a3ecbe5\": []}}, {\"mode\": \"vega-lite\"});\n",
       "</script>"
      ],
      "text/plain": [
       "alt.Chart(...)"
      ]
     },
     "execution_count": 5,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "alt.Chart(final_df.filter(namespace=\"drs-pipelines-dev\")).mark_bar().encode( # type: ignore\n",
    "    x='pod',\n",
    "    y='sum(milli_cpus)',\n",
    "    color='container',\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "90.555"
      ]
     },
     "execution_count": 6,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "\"\"\"Totals\"\"\"\n",
    "total_milli_cpus = final_df.get_column(\"milli_cpus\").sum()\n",
    "total_milli_cpus / 1000"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "\n",
       "<style>\n",
       "  #altair-viz-a387711424c94d8ca41b0f46978400f4.vega-embed {\n",
       "    width: 100%;\n",
       "    display: flex;\n",
       "  }\n",
       "\n",
       "  #altair-viz-a387711424c94d8ca41b0f46978400f4.vega-embed details,\n",
       "  #altair-viz-a387711424c94d8ca41b0f46978400f4.vega-embed details summary {\n",
       "    position: relative;\n",
       "  }\n",
       "</style>\n",
       "<div id=\"altair-viz-a387711424c94d8ca41b0f46978400f4\"></div>\n",
       "<script type=\"text/javascript\">\n",
       "  var VEGA_DEBUG = (typeof VEGA_DEBUG == \"undefined\") ? {} : VEGA_DEBUG;\n",
       "  (function(spec, embedOpt){\n",
       "    let outputDiv = document.currentScript.previousElementSibling;\n",
       "    if (outputDiv.id !== \"altair-viz-a387711424c94d8ca41b0f46978400f4\") {\n",
       "      outputDiv = document.getElementById(\"altair-viz-a387711424c94d8ca41b0f46978400f4\");\n",
       "    }\n",
       "\n",
       "    const paths = {\n",
       "      \"vega\": \"https://cdn.jsdelivr.net/npm/vega@5?noext\",\n",
       "      \"vega-lib\": \"https://cdn.jsdelivr.net/npm/vega-lib?noext\",\n",
       "      \"vega-lite\": \"https://cdn.jsdelivr.net/npm/vega-lite@5.20.1?noext\",\n",
       "      \"vega-embed\": \"https://cdn.jsdelivr.net/npm/vega-embed@6?noext\",\n",
       "    };\n",
       "\n",
       "    function maybeLoadScript(lib, version) {\n",
       "      var key = `${lib.replace(\"-\", \"\")}_version`;\n",
       "      return (VEGA_DEBUG[key] == version) ?\n",
       "        Promise.resolve(paths[lib]) :\n",
       "        new Promise(function(resolve, reject) {\n",
       "          var s = document.createElement('script');\n",
       "          document.getElementsByTagName(\"head\")[0].appendChild(s);\n",
       "          s.async = true;\n",
       "          s.onload = () => {\n",
       "            VEGA_DEBUG[key] = version;\n",
       "            return resolve(paths[lib]);\n",
       "          };\n",
       "          s.onerror = () => reject(`Error loading script: ${paths[lib]}`);\n",
       "          s.src = paths[lib];\n",
       "        });\n",
       "    }\n",
       "\n",
       "    function showError(err) {\n",
       "      outputDiv.innerHTML = `<div class=\"error\" style=\"color:red;\">${err}</div>`;\n",
       "      throw err;\n",
       "    }\n",
       "\n",
       "    function displayChart(vegaEmbed) {\n",
       "      vegaEmbed(outputDiv, spec, embedOpt)\n",
       "        .catch(err => showError(`Javascript Error: ${err.message}<br>This usually means there's a typo in your chart specification. See the javascript console for the full traceback.`));\n",
       "    }\n",
       "\n",
       "    if(typeof define === \"function\" && define.amd) {\n",
       "      requirejs.config({paths});\n",
       "      let deps = [\"vega-embed\"];\n",
       "      require(deps, displayChart, err => showError(`Error loading script: ${err.message}`));\n",
       "    } else {\n",
       "      maybeLoadScript(\"vega\", \"5\")\n",
       "        .then(() => maybeLoadScript(\"vega-lite\", \"5.20.1\"))\n",
       "        .then(() => maybeLoadScript(\"vega-embed\", \"6\"))\n",
       "        .catch(showError)\n",
       "        .then(() => displayChart(vegaEmbed));\n",
       "    }\n",
       "  })({\"config\": {\"view\": {\"continuousWidth\": 300, \"continuousHeight\": 300}}, \"data\": {\"name\": \"data-4995ea3a04c94c6114bd6a684798f199\"}, \"mark\": {\"type\": \"bar\"}, \"encoding\": {\"color\": {\"field\": \"container\", \"type\": \"nominal\"}, \"x\": {\"field\": \"namespace\", \"type\": \"nominal\"}, \"y\": {\"aggregate\": \"sum\", \"field\": \"milli_cpus\", \"type\": \"quantitative\"}}, \"$schema\": \"https://vega.github.io/schema/vega-lite/v5.20.1.json\", \"datasets\": {\"data-4995ea3a04c94c6114bd6a684798f199\": [{\"namespace\": \"acorn\", \"pod\": \"acorn-client-5555d69494-qq8dp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"acorn\", \"pod\": \"acorn-client-5555d69494-qq8dp\", \"container\": \"acorn-client\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"acorn\", \"pod\": \"acorn-client-fc995884b-hnzdz\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"acorn\", \"pod\": \"acorn-client-fc995884b-hnzdz\", \"container\": \"acorn-client\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"acorn\", \"pod\": \"acorn-client-fc995884b-m9fhb\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"acorn\", \"pod\": \"acorn-client-fc995884b-m9fhb\", \"container\": \"acorn-client\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"acorn\", \"pod\": \"acorn-server-55cc8d8d97-x8zdg\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"acorn\", \"pod\": \"acorn-server-55cc8d8d97-x8zdg\", \"container\": \"acorn-server\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"acorn\", \"pod\": \"acorn-server-657f86dfb-4hvlz\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"acorn\", \"pod\": \"acorn-server-657f86dfb-4hvlz\", \"container\": \"acorn-server\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"acorn\", \"pod\": \"acorn-server-657f86dfb-smq2d\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"acorn\", \"pod\": \"acorn-server-657f86dfb-smq2d\", \"container\": \"acorn-server\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"almascripts\", \"pod\": \"request-translator-29415825-6thtp\", \"container\": \"almascripts\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"aqueduct\", \"pod\": \"mystic-9764f6b76-fnngg\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"aqueduct\", \"pod\": \"mystic-9764f6b76-fnngg\", \"container\": \"mystic\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"aqueduct\", \"pod\": \"spicket-667bf88d56-mdqz2\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"aqueduct\", \"pod\": \"winthrop-5d55d959d5-w867l\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"archecker\", \"pod\": \"archecker-567675cf69-x9nvf\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"archecker\", \"pod\": \"archecker-567675cf69-x9nvf\", \"container\": \"archecker\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"archecker\", \"pod\": \"archecker-85dc69cb8-ztbgv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"archecker\", \"pod\": \"archecker-85dc69cb8-ztbgv\", \"container\": \"archecker\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"arclight\", \"pod\": \"arclight-59bb9b68df-ntbxl\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"arclight\", \"pod\": \"arclight-59bb9b68df-ntbxl\", \"container\": \"arclight\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"arclight\", \"pod\": \"arclight-59bb9b68df-ntbxl\", \"container\": \"arclight-proxy\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"arclight\", \"pod\": \"arclight-59bb9b68df-q2smx\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"arclight\", \"pod\": \"arclight-59bb9b68df-q2smx\", \"container\": \"arclight\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"arclight\", \"pod\": \"arclight-59bb9b68df-q2smx\", \"container\": \"arclight-proxy\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"arclight\", \"pod\": \"arclight-aspace-full-n9gkm\", \"container\": \"arclight-aspace-full\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"arclight\", \"pod\": \"arclight-db7cffb64-vqv6h\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"arclight\", \"pod\": \"arclight-db7cffb64-vqv6h\", \"container\": \"arclight\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"arclight\", \"pod\": \"arclight-db7cffb64-vqv6h\", \"container\": \"arclight-proxy\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"arclight\", \"pod\": \"downloads-bucket-5598844d4d-v4pdm\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"arclight\", \"pod\": \"downloads-bucket-5598844d4d-v4pdm\", \"container\": \"downloads-bucket\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"arclight\", \"pod\": \"downloads-bucket-898965d6-n4twq\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"arclight\", \"pod\": \"downloads-bucket-898965d6-n4twq\", \"container\": \"downloads-bucket\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"aspace\", \"pod\": \"aspace-559c8996df-d7dbc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"aspace\", \"pod\": \"aspace-559c8996df-d7dbc\", \"container\": \"aspace\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"aspace\", \"pod\": \"aspace-6f75997556-h2cwq\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"aspace\", \"pod\": \"aspace-6f75997556-h2cwq\", \"container\": \"aspace\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"bibdatalookup\", \"pod\": \"bibdatalookup-6999cfc889-4bmc5\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"bibdatalookup\", \"pod\": \"bibdatalookup-6999cfc889-4bmc5\", \"container\": \"bibdatalookup\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"bibdatalookup\", \"pod\": \"bibdatalookup-c958ffccb-hwtxs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"bibdatalookup\", \"pod\": \"bibdatalookup-c958ffccb-hwtxs\", \"container\": \"bibdatalookup\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"bibdatalookup\", \"pod\": \"bibdatalookup-c958ffccb-thqx5\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"bibdatalookup\", \"pod\": \"bibdatalookup-c958ffccb-thqx5\", \"container\": \"bibdatalookup\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"booklabeler\", \"pod\": \"booklabeler-58f4d6449d-664b8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"booklabeler\", \"pod\": \"booklabeler-58f4d6449d-664b8\", \"container\": \"booklabeler\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"booklabeler\", \"pod\": \"booklabeler-7ccc4cd97b-2k77b\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"booklabeler\", \"pod\": \"booklabeler-7ccc4cd97b-2k77b\", \"container\": \"booklabeler\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"cattle-monitoring-system\", \"pod\": \"alertmanager-rancher-monitoring-alertmanager-0\", \"container\": \"alertmanager\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"cattle-monitoring-system\", \"pod\": \"prometheus-rancher-monitoring-prometheus-0\", \"container\": \"prometheus\", \"cpu_value\": 1250.0, \"cpu_units\": \"m\", \"milli_cpus\": 1.3803765667274033}, {\"namespace\": \"cattle-monitoring-system\", \"pod\": \"rancher-monitoring-grafana-5b76cb7f66-f56lg\", \"container\": \"grafana\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"collex\", \"pod\": \"collex-llm-api-deploy-5dd468985f-h7m74\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"collex\", \"pod\": \"collex-llm-api-deploy-5dd468985f-h7m74\", \"container\": \"collex-llm-api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"collex\", \"pod\": \"collex-llm-api-deploy-5fcd74668-5hvtj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"collex\", \"pod\": \"collex-llm-api-deploy-5fcd74668-5hvtj\", \"container\": \"collex-llm-api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"collex\", \"pod\": \"collex-public-frontend-nextjs-deploy-5f5bf6859c-7wmxp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"collex\", \"pod\": \"collex-public-frontend-nextjs-deploy-5f5bf6859c-7wmxp\", \"container\": \"collex-public-frontend-nextjs\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"collex\", \"pod\": \"collex-public-frontend-nextjs-deploy-84698c8676-dk4kc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"collex\", \"pod\": \"collex-public-frontend-nextjs-deploy-84698c8676-dk4kc\", \"container\": \"collex-public-frontend-nextjs\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"collex\", \"pod\": \"collex-search-api-deploy-5996868f8b-d9dvk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"collex\", \"pod\": \"collex-search-api-deploy-5996868f8b-d9dvk\", \"container\": \"collex-search-api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"collex\", \"pod\": \"collex-search-api-deploy-9b5688597-d6xsb\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"collex\", \"pod\": \"collex-search-api-deploy-9b5688597-d6xsb\", \"container\": \"collex-search-api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"curiosity\", \"pod\": \"curiosity-68fdbbd97c-c749x\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"curiosity\", \"pod\": \"curiosity-68fdbbd97c-c749x\", \"container\": \"curiosity\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"curiosity\", \"pod\": \"curiosity-7f6cdf79d8-86tr8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"curiosity\", \"pod\": \"curiosity-7f6cdf79d8-86tr8\", \"container\": \"curiosity\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"curiosity\", \"pod\": \"curiosity-7f6cdf79d8-gr7hj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"curiosity\", \"pod\": \"curiosity-7f6cdf79d8-gr7hj\", \"container\": \"curiosity\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"curiosity\", \"pod\": \"curiosityworker-89f455446-cv854\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"curiosity\", \"pod\": \"curiosityworker-89f455446-cv854\", \"container\": \"curiosityworker\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"dais\", \"pod\": \"dims-6c9b9654cc-f8n64\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"dais\", \"pod\": \"dims-6c9b9654cc-f8n64\", \"container\": \"dims\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"dais\", \"pod\": \"dims-7597788cc-zjn54\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"dais\", \"pod\": \"dims-7597788cc-zjn54\", \"container\": \"dims\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"eda\", \"pod\": \"eda-57bbcd456c-9mhml\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"eda\", \"pod\": \"eda-57bbcd456c-9mhml\", \"container\": \"eda\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"eda\", \"pod\": \"eda-57bbcd456c-9mhml\", \"container\": \"eda-proxy\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"eda\", \"pod\": \"eda-645d6bb5d5-7shhr\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"eda\", \"pod\": \"eda-645d6bb5d5-7shhr\", \"container\": \"eda\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"eda\", \"pod\": \"eda-645d6bb5d5-7shhr\", \"container\": \"eda-proxy\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"eda\", \"pod\": \"eda-iip-668f88f747-9nv4c\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"eda\", \"pod\": \"eda-iip-668f88f747-9nv4c\", \"container\": \"eda-iip\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"eda\", \"pod\": \"eda-iip-f9cfdf4ff-n4qs6\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"eda\", \"pod\": \"eda-iip-f9cfdf4ff-n4qs6\", \"container\": \"eda-iip\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"eda\", \"pod\": \"eda-mc-69b5dfd9b7-7krxw\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"eda\", \"pod\": \"eda-mc-69b5dfd9b7-7krxw\", \"container\": \"eda-mc\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fal-small-images\", \"pod\": \"fal-small-images-5db99dbf4d-7gsvd\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fal-small-images\", \"pod\": \"fal-small-images-5db99dbf4d-7gsvd\", \"container\": \"fal-small-images\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fal-small-images\", \"pod\": \"fal-small-images-6f9fdbc578-x8f2t\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fal-small-images\", \"pod\": \"fal-small-images-6f9fdbc578-x8f2t\", \"container\": \"fal-small-images\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"api-6d688d66b9-2t4rv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"api-6d688d66b9-2t4rv\", \"container\": \"api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"api-85cdbb4745-lc2zc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"api-85cdbb4745-lc2zc\", \"container\": \"api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"api-85cdbb4745-szkq8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"api-85cdbb4745-szkq8\", \"container\": \"api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"loader-5597fc5866-5rwxn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"loader-5597fc5866-5rwxn\", \"container\": \"loader\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"loader-cb8db4956-72n9s\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"loader-cb8db4956-72n9s\", \"container\": \"loader\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"loader-cb8db4956-ffcng\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"loader-cb8db4956-ffcng\", \"container\": \"loader\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"proxy-648fd58784-2lh5p\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"proxy-648fd58784-2lh5p\", \"container\": \"proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"proxy-6d65bb7b4d-lgnv5\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"proxy-6d65bb7b4d-lgnv5\", \"container\": \"proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"ui-5f6bf67c85-mjpg2\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"ui-5f6bf67c85-mjpg2\", \"container\": \"ui\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"ui-646d7b6848-hggrl\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"ui-646d7b6848-hggrl\", \"container\": \"ui\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"ui-646d7b6848-qxkgw\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"fts\", \"pod\": \"ui-646d7b6848-qxkgw\", \"container\": \"ui\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"hgl\", \"pod\": \"gbl-758879d5c9-bphvt\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"hgl\", \"pod\": \"gbl-758879d5c9-bphvt\", \"container\": \"gbl\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"hgl\", \"pod\": \"gbl-944b859d5-fkfqs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"hgl\", \"pod\": \"gbl-944b859d5-fkfqs\", \"container\": \"gbl\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"hgl\", \"pod\": \"gbl-944b859d5-g5mnc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"hgl\", \"pod\": \"gbl-944b859d5-g5mnc\", \"container\": \"gbl\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"hgl\", \"pod\": \"geoserver-6f8b474798-pvmms\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"hgl\", \"pod\": \"geoserver-6f8b474798-pvmms\", \"container\": \"geoserver\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"hgl\", \"pod\": \"geoserver-6f8b474798-pvmms\", \"container\": \"hgl-proxy\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"hgl\", \"pod\": \"geoserver-86b547465d-8q9g9\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"hgl\", \"pod\": \"geoserver-86b547465d-8q9g9\", \"container\": \"geoserver\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"hgl\", \"pod\": \"geoserver-86b547465d-8q9g9\", \"container\": \"hgl-proxy\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"ids\", \"pod\": \"ids-56f564f986-7sbgk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"ids-56f564f986-7sbgk\", \"container\": \"ids\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"ids-68656d5c55-cbjnn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"ids-68656d5c55-cbjnn\", \"container\": \"ids\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"ids-68656d5c55-jss9c\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"ids-68656d5c55-jss9c\", \"container\": \"ids\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"idsmd-6d76f7f9b4-p6dcq\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"idsmd-6d76f7f9b4-p6dcq\", \"container\": \"idsmd\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"idsmd-6d76f7f9b4-qplhk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"idsmd-6d76f7f9b4-qplhk\", \"container\": \"idsmd\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"idsmd-6dfbc9cfbd-9xjc7\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"idsmd-6dfbc9cfbd-9xjc7\", \"container\": \"idsmd\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"iiif-5744dc767f-5ckvv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"iiif-5744dc767f-5ckvv\", \"container\": \"iiif\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"iiif-7b47cf6d9-lqkgv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"iiif-7b47cf6d9-lqkgv\", \"container\": \"iiif\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"iiif-7b47cf6d9-m5jjh\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"iiif-7b47cf6d9-m5jjh\", \"container\": \"iiif\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"iiif-updater-7557ccccff-bwgcf\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"iiif-updater-7557ccccff-bwgcf\", \"container\": \"iiif-updater\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"pds-f58f57594-8cwls\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"pds-f58f57594-8cwls\", \"container\": \"pds\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"pds-f58f57594-ks495\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"ids\", \"pod\": \"pds-f58f57594-ks495\", \"container\": \"pds\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"imgconv\", \"pod\": \"imgconv-84cb96d846-88cpx\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"imgconv\", \"pod\": \"imgconv-84cb96d846-88cpx\", \"container\": \"imgconv\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.13803765667274032}, {\"namespace\": \"imgconv\", \"pod\": \"imgconv-84cb96d846-zg2mk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"imgconv\", \"pod\": \"imgconv-84cb96d846-zg2mk\", \"container\": \"imgconv\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.13803765667274032}, {\"namespace\": \"istio-system\", \"pod\": \"istio-apiprivate-ingressgateway-7c4bd499d9-7lctf\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"istio-system\", \"pod\": \"istio-ingressgateway-57c7fdd696-rdx95\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"istio-system\", \"pod\": \"istio-private-ingressgateway-7b69c99dc-sh8h8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"istio-system\", \"pod\": \"istio-server-ingressgateway-66dd9898b9-6rngs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"istio-system\", \"pod\": \"istiod-84cd86b54d-vrkvc\", \"container\": \"discovery\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"istio-system\", \"pod\": \"jaeger-7799f58d6-7h4n2\", \"container\": \"jaegertracing\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"istio-system\", \"pod\": \"kiali-59cc5b5f4d-jsf4f\", \"container\": \"kiali\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"jobmonitor\", \"pod\": \"jobmonitor-d597f6ddf-7fqkc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jobmonitor\", \"pod\": \"jobmonitor-d597f6ddf-7fqkc\", \"container\": \"jobmonitor\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"jobmonitor\", \"pod\": \"jobmonitor-d597f6ddf-cfsrj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jobmonitor\", \"pod\": \"jobmonitor-d597f6ddf-cfsrj\", \"container\": \"jobmonitor\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"jstor\", \"pod\": \"downloads-bucket-ead-7cb584c88b-r669w\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"downloads-bucket-ead-7cb584c88b-r669w\", \"container\": \"downloads-bucket-ead\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"jstor\", \"pod\": \"downloads-bucket-via-594bdb8fbd-44br7\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"downloads-bucket-via-594bdb8fbd-44br7\", \"container\": \"downloads-bucket-via\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"jstor\", \"pod\": \"jstor-aggregator-6cc4f4ff5b-jhw9h\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"jstor-aggregator-6cc4f4ff5b-jhw9h\", \"container\": \"jstor-aggregator\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"jstor\", \"pod\": \"jstor-aggregator-6cc4f4ff5b-vd4n4\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"jstor-aggregator-6cc4f4ff5b-vd4n4\", \"container\": \"jstor-aggregator\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"jstor\", \"pod\": \"jstor-aggregator-retro-56b74fdc4-hvbz8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"jstor-aggregator-retro-56b74fdc4-hvbz8\", \"container\": \"jstor-aggregator-retro\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"jstor\", \"pod\": \"jstor-harvester-759467f6f6-h247h\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"jstor-harvester-759467f6f6-h247h\", \"container\": \"jstor-harvester\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"jstor\", \"pod\": \"jstor-harvester-759467f6f6-trpdx\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"jstor-harvester-759467f6f6-trpdx\", \"container\": \"jstor-harvester\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"jstor\", \"pod\": \"jstor-harvester-retro-f9d86ffbf-5hq6x\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"jstor-harvester-retro-f9d86ffbf-5hq6x\", \"container\": \"jstor-harvester-retro\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"jstor\", \"pod\": \"jstor-integration-55ccf5795d-7tn5j\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"jstor-integration-55ccf5795d-7tn5j\", \"container\": \"jstor-integration\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"jstor\", \"pod\": \"jstor-integration-55ccf5795d-fgblc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"jstor-integration-55ccf5795d-fgblc\", \"container\": \"jstor-integration\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"jstor\", \"pod\": \"jstor-publisher-79996b59f-hltwz\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"jstor-publisher-79996b59f-hltwz\", \"container\": \"jstor-publisher\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"jstor\", \"pod\": \"jstor-publisher-79996b59f-j5dqd\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"jstor-publisher-79996b59f-j5dqd\", \"container\": \"jstor-publisher\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"jstor\", \"pod\": \"jstor-publisher-retro-76d9bb94d-2x2k8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"jstor-publisher-retro-76d9bb94d-2x2k8\", \"container\": \"jstor-publisher-retro\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"jstor\", \"pod\": \"jstor-transformer-5ff7b76798-4c2fp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"jstor-transformer-5ff7b76798-4c2fp\", \"container\": \"jstor-transformer\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"jstor\", \"pod\": \"jstor-transformer-5ff7b76798-qgjwc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"jstor-transformer-5ff7b76798-qgjwc\", \"container\": \"jstor-transformer\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"jstor\", \"pod\": \"jstor-transformer-retro-56d449ff99-tcp2r\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"jstor\", \"pod\": \"jstor-transformer-retro-56d449ff99-tcp2r\", \"container\": \"jstor-transformer-retro\", \"cpu_value\": 500.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.5521506266909613}, {\"namespace\": \"kafka\", \"pod\": \"lts-kafka-dual-role-0\", \"container\": \"kafka\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"kafka\", \"pod\": \"lts-kafka-dual-role-1\", \"container\": \"kafka\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"kafka\", \"pod\": \"lts-kafka-dual-role-2\", \"container\": \"kafka\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"kafka\", \"pod\": \"lts-kafka-entity-operator-c464ccbcc-8xd4k\", \"container\": \"topic-operator\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"kafka\", \"pod\": \"lts-kafka-entity-operator-c464ccbcc-8xd4k\", \"container\": \"user-operator\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"kafka\", \"pod\": \"strimzi-cluster-operator-59bb568c45-vcg8h\", \"container\": \"strimzi-cluster-operator\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.22086025067638448}, {\"namespace\": \"kube-system\", \"pod\": \"cloud-controller-manager-ip-10-137-166-22\", \"container\": \"cloud-controller-manager\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"cloud-controller-manager-ip-10-137-166-26\", \"container\": \"cloud-controller-manager\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"cloud-controller-manager-ip-10-137-166-27\", \"container\": \"cloud-controller-manager\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-controller-69dc5b4c8c-4cxt5\", \"container\": \"csi-provisioner\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-controller-69dc5b4c8c-4cxt5\", \"container\": \"csi-resizer\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-controller-69dc5b4c8c-4cxt5\", \"container\": \"csi-snapshotter\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-controller-69dc5b4c8c-4cxt5\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-controller-69dc5b4c8c-4cxt5\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-7ttzk\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-7ttzk\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-7ttzk\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-8ntsn\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-8ntsn\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-8ntsn\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-9rxl4\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-9rxl4\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-9rxl4\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-cvmrw\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-cvmrw\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-cvmrw\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-hcrps\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-hcrps\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-hcrps\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-khqwx\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-khqwx\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-khqwx\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-m8fws\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-m8fws\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-m8fws\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-wcm6w\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-wcm6w\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-wcm6w\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-wdpbm\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-wdpbm\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"csi-nfs-node-wdpbm\", \"container\": \"nfs\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-q7bpm\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-q7bpm\", \"container\": \"csi-provisioner\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-q7bpm\", \"container\": \"csi-attacher\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-q7bpm\", \"container\": \"csi-snapshotter\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-q7bpm\", \"container\": \"csi-resizer\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-q7bpm\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-wstjf\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-wstjf\", \"container\": \"csi-provisioner\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-wstjf\", \"container\": \"csi-attacher\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-wstjf\", \"container\": \"csi-snapshotter\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-wstjf\", \"container\": \"csi-resizer\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-controller-7d47fbcdc9-wstjf\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-9wptm\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-9wptm\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-9wptm\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-9zb49\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-9zb49\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-9zb49\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-dtmqn\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-dtmqn\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-dtmqn\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-gk7wz\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-gk7wz\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-gk7wz\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-j58cz\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-j58cz\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-j58cz\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-jrb29\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-jrb29\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-jrb29\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-m8tlx\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-m8tlx\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-m8tlx\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-v9wgj\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-v9wgj\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-v9wgj\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-zl2bj\", \"container\": \"ebs-plugin\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-zl2bj\", \"container\": \"node-driver-registrar\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"ebs-csi-node-zl2bj\", \"container\": \"liveness-probe\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"kube-system\", \"pod\": \"etcd-ip-10-137-166-22\", \"container\": \"etcd\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.22086025067638448}, {\"namespace\": \"kube-system\", \"pod\": \"etcd-ip-10-137-166-26\", \"container\": \"etcd\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.22086025067638448}, {\"namespace\": \"kube-system\", \"pod\": \"etcd-ip-10-137-166-27\", \"container\": \"etcd\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.22086025067638448}, {\"namespace\": \"kube-system\", \"pod\": \"kube-apiserver-ip-10-137-166-22\", \"container\": \"kube-apiserver\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"kube-apiserver-ip-10-137-166-26\", \"container\": \"kube-apiserver\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"kube-apiserver-ip-10-137-166-27\", \"container\": \"kube-apiserver\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"kube-controller-manager-ip-10-137-166-22\", \"container\": \"kube-controller-manager\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.22086025067638448}, {\"namespace\": \"kube-system\", \"pod\": \"kube-controller-manager-ip-10-137-166-26\", \"container\": \"kube-controller-manager\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.22086025067638448}, {\"namespace\": \"kube-system\", \"pod\": \"kube-controller-manager-ip-10-137-166-27\", \"container\": \"kube-controller-manager\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.22086025067638448}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-166-22\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-166-26\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-166-27\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-167-20\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-167-22\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-167-24\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-167-28\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-167-29\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"kube-proxy-ip-10-137-167-30\", \"container\": \"kube-proxy\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"kube-scheduler-ip-10-137-166-22\", \"container\": \"kube-scheduler\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"kube-scheduler-ip-10-137-166-26\", \"container\": \"kube-scheduler\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"kube-scheduler-ip-10-137-166-27\", \"container\": \"kube-scheduler\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-582tl\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-5hxqk\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-7k9m5\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-gqpxz\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-lxjqn\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-r4slr\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-srzln\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-tqd47\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-canal-v8nt6\", \"container\": \"calico-node\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-coredns-rke2-coredns-85d6696775-ph7v7\", \"container\": \"coredns\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-coredns-rke2-coredns-85d6696775-q9clm\", \"container\": \"coredns\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-coredns-rke2-coredns-autoscaler-665b7f6f86-rtrl7\", \"container\": \"autoscaler\", \"cpu_value\": 25.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.02760753133454806}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-5d7xf\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-9dzcz\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-d7qt6\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-hh6hw\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-hxx5p\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-k5j7n\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-nkh47\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-sjqbh\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-ingress-nginx-controller-v6sl6\", \"container\": \"rke2-ingress-nginx-controller\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"kube-system\", \"pod\": \"rke2-metrics-server-7c4c577547-9zj5g\", \"container\": \"metrics-server\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-collections-547756785c-pnt49\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-collections-547756785c-pnt49\", \"container\": \"lc-collections\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-collections-547756785c-vld6j\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-collections-547756785c-vld6j\", \"container\": \"lc-collections\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-collectionsbuilder-7c8bd8575-mz8hx\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-collectionsbuilder-7c8bd8575-mz8hx\", \"container\": \"lc-collectionsbuilder\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-control-server-75b8bfc566-pckkn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-control-server-75b8bfc566-pckkn\", \"container\": \"lc-control-server\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-csv-df57cbc58-gg4w9\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-csv-df57cbc58-gg4w9\", \"container\": \"lc-csv\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-csv-df57cbc58-z6wgv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-csv-df57cbc58-z6wgv\", \"container\": \"lc-csv\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-ingest-868b55594f-2gkjn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-ingest-868b55594f-2gkjn\", \"container\": \"lc-ingest\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-ingest-868b55594f-r6tqf\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-ingest-868b55594f-r6tqf\", \"container\": \"lc-ingest\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-oai-69887f59b5-rk5hs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-oai-69887f59b5-rk5hs\", \"container\": \"lc-oai\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-oai-69887f59b5-vlclq\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"lc-oai-69887f59b5-vlclq\", \"container\": \"lc-oai\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"librarycloud-694f6dfdbb-r8htx\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"librarycloud\", \"pod\": \"librarycloud-694f6dfdbb-r8htx\", \"container\": \"librarycloud\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"listview\", \"pod\": \"listview-7bc65f4c45-4wxwh\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"listview\", \"pod\": \"listview-7bc65f4c45-4wxwh\", \"container\": \"listview\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"listview\", \"pod\": \"listview-7bc65f4c45-kxk44\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"listview\", \"pod\": \"listview-7bc65f4c45-kxk44\", \"container\": \"listview\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"listview\", \"pod\": \"listview-md-65f9569858-cwv8f\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"listview\", \"pod\": \"listview-md-65f9569858-cwv8f\", \"container\": \"listview-md\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"loki\", \"pod\": \"loki-0\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"loki\", \"pod\": \"loki-grafana-8cd994887-flltj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-762cs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-98b89\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-b84px\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-blcb5\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-mgbjp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-mnf7j\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-nhz77\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-pq6b5\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"loki\", \"pod\": \"loki-promtail-z2hjv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"lts-pipelines\", \"pod\": \"collex-main-ingest-pipeline-finalize-mongo-job-acqpkr6m\", \"container\": \"base\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-pgbouncer-deploy-dbdd6cdf7-8zdj8\", \"container\": \"pgbouncer\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-scheduler-deploy-6c9f65c84c-f6wb6\", \"container\": \"scheduler\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-scheduler-deploy-6c9f65c84c-qmddr\", \"container\": \"scheduler\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-triggerer-deploy-799c4ddc47-kcshl\", \"container\": \"triggerer\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-webserver-deploy-cb459dc58-5bjpv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-webserver-deploy-cb459dc58-5bjpv\", \"container\": \"webserver\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-webserver-deploy-cb459dc58-x4zfp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"lts-pipelines\", \"pod\": \"lts-pipelines-airflow-webserver-deploy-cb459dc58-x4zfp\", \"container\": \"webserver\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mds\", \"pod\": \"mdsrv-6b9b5568b4-nnkvl\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mds\", \"pod\": \"mdsrv-6b9b5568b4-nnkvl\", \"container\": \"mdsrv\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.13803765667274032}, {\"namespace\": \"mds\", \"pod\": \"mdsrv-6b9b5568b4-v4pkz\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mds\", \"pod\": \"mdsrv-6b9b5568b4-v4pkz\", \"container\": \"mdsrv\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.13803765667274032}, {\"namespace\": \"mds\", \"pod\": \"mqsrv-7f44bc5b4f-dqn9h\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mds\", \"pod\": \"mqsrv-7f44bc5b4f-dqn9h\", \"container\": \"mqsrv\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.13803765667274032}, {\"namespace\": \"mds\", \"pod\": \"mqsrv-7f44bc5b4f-v8wtk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mds\", \"pod\": \"mqsrv-7f44bc5b4f-v8wtk\", \"container\": \"mqsrv\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.13803765667274032}, {\"namespace\": \"mds\", \"pod\": \"ocfl-storage-ro-5fb6594c45-8jgzn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mds\", \"pod\": \"ocfl-storage-ro-5fb6594c45-8jgzn\", \"container\": \"ocfl-storage-ro\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.13803765667274032}, {\"namespace\": \"mds\", \"pod\": \"s3pather-675d9bbcdb-46znp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mds\", \"pod\": \"s3pather-675d9bbcdb-46znp\", \"container\": \"s3pather\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.13803765667274032}, {\"namespace\": \"mds\", \"pod\": \"s3pather-675d9bbcdb-crcj5\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mds\", \"pod\": \"s3pather-675d9bbcdb-crcj5\", \"container\": \"s3pather\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.13803765667274032}, {\"namespace\": \"mds\", \"pod\": \"solr2mds-64f8bbb966-8fvsc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mds\", \"pod\": \"solr2mds-64f8bbb966-8fvsc\", \"container\": \"solr2mds\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.13803765667274032}, {\"namespace\": \"mds\", \"pod\": \"solr2mds-64f8bbb966-hfjtk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mds\", \"pod\": \"solr2mds-64f8bbb966-hfjtk\", \"container\": \"solr2mds\", \"cpu_value\": 125.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.13803765667274032}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-drs-gateway-b595f977c-sj4jt\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-drs-gateway-b595f977c-sj4jt\", \"container\": \"md-operational-drs-gateway\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-mongo-file-ingestor-6cff445458-q5rvt\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-mongo-file-ingestor-6cff445458-q5rvt\", \"container\": \"md-operational-mongo-file-ingestor\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-mongo-ingestor-76fb97fc65-d6bvt\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-mongo-ingestor-76fb97fc65-d6bvt\", \"container\": \"md-operational-mongo-ingestor\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-solr-ingestor-566dcfbbb-6dqt9\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"metadata-operational\", \"pod\": \"md-operational-solr-ingestor-566dcfbbb-6dqt9\", \"container\": \"md-operational-solr-ingestor\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-embed-655d9bff6d-d6rqn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-embed-655d9bff6d-d6rqn\", \"container\": \"mps-embed\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-embed-655d9bff6d-tdnpw\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-embed-655d9bff6d-tdnpw\", \"container\": \"mps-embed\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-viewer-57594b8f58-8b4tt\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-viewer-57594b8f58-8b4tt\", \"container\": \"mps-viewer\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-viewer-57594b8f58-hb8k8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps-viewer\", \"pod\": \"mps-viewer-57594b8f58-hb8k8\", \"container\": \"mps-viewer\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"mps\", \"pod\": \"access-service-c7fd6b5cf-6zvn6\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"access-service-c7fd6b5cf-6zvn6\", \"container\": \"access-service\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"asset-cache-mgr-577fcd89fd-74txv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"asset-cache-mgr-577fcd89fd-74txv\", \"container\": \"asset-cache-mgr\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"asset-db-connector-c55b55685-2drcw\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"asset-db-connector-c55b55685-2drcw\", \"container\": \"asset-db-connector\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"asset-delivery-files-7f7b94777b-ccf4p\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"asset-delivery-files-7f7b94777b-ccf4p\", \"container\": \"asset-delivery-files\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"asset-delivery-images-8589bf4769-fvm2g\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"asset-delivery-images-8589bf4769-fvm2g\", \"container\": \"asset-delivery-images\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"asset-delivery-thumbnails-b98548df-rnp2c\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"asset-delivery-thumbnails-b98548df-rnp2c\", \"container\": \"asset-delivery-thumbnails\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"authorization-6ffbfdf8ff-t9npw\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"authorization-6ffbfdf8ff-t9npw\", \"container\": \"authorization\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"cache-populator-85b8b5fb9d-bvtbd\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"cache-populator-85b8b5fb9d-bvtbd\", \"container\": \"cache-populator\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-aws-kak-d4d77787b-hqb7p\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-aws-kak-d4d77787b-hqb7p\", \"container\": \"cantaloupe-aws-kak\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-aws-ojpg-6ff6fc5d67-gh9bs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-aws-ojpg-6ff6fc5d67-gh9bs\", \"container\": \"cantaloupe-aws-ojpg\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-proxy-b894d47f6-s4pjt\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-proxy-b894d47f6-s4pjt\", \"container\": \"cantaloupe-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-was-kak-74f9979b6f-7ffz8\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-was-kak-74f9979b6f-7ffz8\", \"container\": \"cantaloupe-was-kak\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-was-ojpg-bcdcbff99-pjjvf\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"cantaloupe-was-ojpg-bcdcbff99-pjjvf\", \"container\": \"cantaloupe-was-ojpg\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"citation-api-db84f7486-w8t5v\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"citation-api-db84f7486-w8t5v\", \"container\": \"citation-api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"deliverys3cache-979f7658-8xkjs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"deliverys3cache-979f7658-8xkjs\", \"container\": \"deliverys3cache\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"deliverys3proxy-56b9db6c4-lvhsn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"deliverys3proxy-56b9db6c4-lvhsn\", \"container\": \"deliverys3proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"ids-redirects-564c7d86c8-94mkk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"ids-redirects-564c7d86c8-94mkk\", \"container\": \"ids-redirects\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"iiif-converter-559d794756-sg9x4\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"iiif-converter-559d794756-sg9x4\", \"container\": \"iiif-converter\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"iiif-presentation-688b56566-fj86j\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"iiif-presentation-688b56566-fj86j\", \"container\": \"iiif-presentation\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"iiif-synchronizer-updater-6cdd57d955-v7798\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"iiif-synchronizer-updater-6cdd57d955-v7798\", \"container\": \"iiif-synchronizer-updater\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"inlib-5c8bb7558f-g9gr6\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"inlib-5c8bb7558f-g9gr6\", \"container\": \"inlib\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"login-8558f4c44f-l2rb4\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"login-8558f4c44f-l2rb4\", \"container\": \"login\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"lookup-58796fdc47-x2mzf\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"lookup-58796fdc47-x2mzf\", \"container\": \"lookup\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"ocrsearch-58876c7b69-55q2r\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"ocrsearch-58876c7b69-55q2r\", \"container\": \"ocrsearch\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"pdf-download-6756c5567c-lw7vq\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"pdf-download-6756c5567c-lw7vq\", \"container\": \"pdf-download\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"url-redirects-86666f67b7-qhg79\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mps\", \"pod\": \"url-redirects-86666f67b7-qhg79\", \"container\": \"url-redirects\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-cache-mgr-67c44b5c54-gltfs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-cache-mgr-67c44b5c54-gltfs\", \"container\": \"asset-cache-mgr\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-copy-7f8d5b456b-h6q22\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-copy-7f8d5b456b-h6q22\", \"container\": \"asset-copy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-db-connector-54969d4596-tzppr\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-db-connector-54969d4596-tzppr\", \"container\": \"asset-db-connector\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-validation-85488cf799-jkb4v\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"asset-validation-85488cf799-jkb4v\", \"container\": \"asset-validation\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"authorization-5d9fd8cd78-lrs77\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"authorization-5d9fd8cd78-lrs77\", \"container\": \"authorization\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"deactivator-7cf7d8cc5-9v6w9\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"deactivator-7cf7d8cc5-9v6w9\", \"container\": \"deactivator\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-converter-6496d46b75-4rkd9\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-converter-6496d46b75-4rkd9\", \"container\": \"iiif-converter\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-metadata-updater-5c46b6877d-79pvn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-metadata-updater-5c46b6877d-79pvn\", \"container\": \"iiif-metadata-updater\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-services-cdc49965d-8l2l5\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-services-cdc49965d-8l2l5\", \"container\": \"iiif-services\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-synchronizer-iterator-64cf46bb55-thns9\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-synchronizer-iterator-64cf46bb55-thns9\", \"container\": \"iiif-synchronizer-iterator\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-synchronizer-updater-5f555df75d-qx7lj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-synchronizer-updater-5f555df75d-qx7lj\", \"container\": \"iiif-synchronizer-updater\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-task-manager-6b598c4fcd-nrrcd\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-task-manager-6b598c4fcd-nrrcd\", \"container\": \"iiif-task-manager\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-validator-f77d96fdc-jn427\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"iiif-validator-f77d96fdc-jn427\", \"container\": \"iiif-validator\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"ingest-6c656dff57-8lv8c\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"ingest-6c656dff57-8lv8c\", \"container\": \"ingest\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"ltsfileservice-776768896d-6wjcd\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"ltsfileservice-776768896d-6wjcd\", \"container\": \"ltsfileservice\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"mpscron-29415859-562kn\", \"container\": \"mpscron\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"mpscron-29415860-pq8fq\", \"container\": \"mpscron\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"multi-asset-ingest-worker-f9fc86888-dfwjb\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"multi-asset-ingest-worker-f9fc86888-dfwjb\", \"container\": \"multi-asset-ingest-worker\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"queue-monitor-5897c74948-skctx\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"mpsadm\", \"pod\": \"queue-monitor-5897c74948-skctx\", \"container\": \"queue-monitor\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"nrs\", \"pod\": \"resolver-7d98f98bb5-7hpwv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"nrs\", \"pod\": \"resolver-7d98f98bb5-7hpwv\", \"container\": \"resolver\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"nrs\", \"pod\": \"resolver-cache-5bf6877c57-24566\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"nrs\", \"pod\": \"resolver-cache-5bf6877c57-24566\", \"container\": \"resolver-cache\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"nrsadm\", \"pod\": \"cache-refresh-29407860-5fhwr\", \"container\": \"cache-refresh\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"nrsadm\", \"pod\": \"cachemgr-7dc594c44f-2cqzk\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"nrsadm\", \"pod\": \"cachemgr-7dc594c44f-2cqzk\", \"container\": \"cachemgr\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"nrsadm\", \"pod\": \"drs-file-urn-29415720-97npj\", \"container\": \"drs-file-urn\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"nrsadm\", \"pod\": \"drs-file-urn-29415780-4l5xw\", \"container\": \"drs-file-urn\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"nrsadm\", \"pod\": \"drs-file-urn-29415840-bkbtz\", \"container\": \"drs-file-urn\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"nrsadm\", \"pod\": \"nrs-admin-api-796f6c64f6-b4vtn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"nrsadm\", \"pod\": \"nrs-admin-api-796f6c64f6-b4vtn\", \"container\": \"nrs-admin-api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"nrsadm\", \"pod\": \"nrs-admin-ui-api-79b69dd46-zgn86\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"nrsadm\", \"pod\": \"nrs-admin-ui-api-79b69dd46-zgn86\", \"container\": \"nrs-admin-ui-api\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"nrsadm\", \"pod\": \"nrs-admin-ui-client-5fc468ff74-jwkjb\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"nrsadm\", \"pod\": \"nrs-admin-ui-client-5fc468ff74-jwkjb\", \"container\": \"nrs-admin-ui-client\", \"cpu_value\": 10.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.011043012533819226}, {\"namespace\": \"olivia\", \"pod\": \"olivia-6998686d85-hkfcj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"olivia\", \"pod\": \"olivia-6998686d85-hkfcj\", \"container\": \"olivia\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"olivia\", \"pod\": \"olivia-md-768f8b66f-2k8zj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"olivia\", \"pod\": \"olivia-md-768f8b66f-2k8zj\", \"container\": \"olivia-md\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"policy\", \"pod\": \"policy-admin-6ccb9bbbd6-6gvnt\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"policy\", \"pod\": \"policy-admin-6ccb9bbbd6-6gvnt\", \"container\": \"policy-admin\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"policy\", \"pod\": \"policy-admin-6ccb9bbbd6-nm2kj\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"policy\", \"pod\": \"policy-admin-6ccb9bbbd6-nm2kj\", \"container\": \"policy-admin\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"post-drs-msg\", \"pod\": \"delivery-file-848848548b-kw9qh\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"post-drs-msg\", \"pod\": \"delivery-file-848848548b-kw9qh\", \"container\": \"delivery-file\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notification-dispatcher-f785b5b66-6tjtg\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notification-dispatcher-f785b5b66-6tjtg\", \"container\": \"drs-notification-dispatcher\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notification-worker-fd5f9b59-2pndn\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notification-worker-fd5f9b59-2pndn\", \"container\": \"drs-notification-worker\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notifier-7988b89fd5-pdp8w\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notifier-7988b89fd5-pdp8w\", \"container\": \"drs-notifier\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notifier-listener-86bc4b4bf5-9nvnb\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"post-drs-msg\", \"pod\": \"drs-notifier-listener-86bc4b4bf5-9nvnb\", \"container\": \"drs-notifier-listener\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"post-drs-msg\", \"pod\": \"lts-pipeline-external-queue-77ff4fc975-6kxzc\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"post-drs-msg\", \"pod\": \"lts-pipeline-external-queue-77ff4fc975-6kxzc\", \"container\": \"lts-pipeline-external-queue\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"post-drs-msg\", \"pod\": \"mps-stager-57945556f6-2cfxs\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"post-drs-msg\", \"pod\": \"mps-stager-57945556f6-2cfxs\", \"container\": \"mps-stager\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"post-drs-msg\", \"pod\": \"nrs-update-755766d4f6-zjl5c\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"post-drs-msg\", \"pod\": \"nrs-update-755766d4f6-zjl5c\", \"container\": \"nrs-update\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"post-drs-msg\", \"pod\": \"sds-delivery-ingest-845cdc786f-6dbck\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"post-drs-msg\", \"pod\": \"sds-delivery-ingest-845cdc786f-6dbck\", \"container\": \"sds-delivery-ingest\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"presto\", \"pod\": \"presto-5c6b88f4c8-qgv49\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"presto\", \"pod\": \"presto-5c6b88f4c8-qgv49\", \"container\": \"presto\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"presto\", \"pod\": \"presto-5c6b88f4c8-t64vp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"presto\", \"pod\": \"presto-5c6b88f4c8-t64vp\", \"container\": \"presto\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"sds\", \"pod\": \"metadata-7779554449-d5klv\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"sds\", \"pod\": \"metadata-7779554449-d5klv\", \"container\": \"metadata\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"sds\", \"pod\": \"sds-85db5db78f-74lkd\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"sds\", \"pod\": \"sds-85db5db78f-74lkd\", \"container\": \"sds\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"sds\", \"pod\": \"wowza-c4ffc4c6d-9zgt9\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"sds\", \"pod\": \"wowza-c4ffc4c6d-9zgt9\", \"container\": \"wowza\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"secureexit\", \"pod\": \"secureexit-5c5959667b-lrrm7\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"secureexit\", \"pod\": \"secureexit-5c5959667b-lrrm7\", \"container\": \"secureexit\", \"cpu_value\": 1.0, \"cpu_units\": null, \"milli_cpus\": 1.1043012533819225}, {\"namespace\": \"shared-shelf-linker\", \"pod\": \"sharedshelf-7c88d954ff-2jxqp\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"shared-shelf-linker\", \"pod\": \"sharedshelf-7c88d954ff-2jxqp\", \"container\": \"sharedshelf\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"solr\", \"pod\": \"solr-qa-solrcloud-zookeeper-0\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"solr\", \"pod\": \"solr-qa-solrcloud-zookeeper-0\", \"container\": \"zookeeper\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"solr\", \"pod\": \"solr-qa-solrcloud-zookeeper-1\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"solr\", \"pod\": \"solr-qa-solrcloud-zookeeper-1\", \"container\": \"zookeeper\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"solr\", \"pod\": \"solr-qa-solrcloud-zookeeper-2\", \"container\": \"istio-proxy\", \"cpu_value\": 100.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.11043012533819224}, {\"namespace\": \"solr\", \"pod\": \"solr-qa-solrcloud-zookeeper-2\", \"container\": \"zookeeper\", \"cpu_value\": 250.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.27607531334548063}, {\"namespace\": \"testkube\", \"pod\": \"testkube-api-server-86b89d59fc-sr2jx\", \"container\": \"testkube-api\", \"cpu_value\": 200.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.22086025067638448}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-api-5d6c7db85b-dhlm2\", \"container\": \"testkube-cloud-api\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.05521506266909612}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-dex-665b57765c-khls9\", \"container\": \"dex\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.05521506266909612}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-nats-0\", \"container\": \"nats\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.05521506266909612}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-nats-0\", \"container\": \"reloader\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.05521506266909612}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-nats-1\", \"container\": \"nats\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.05521506266909612}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-nats-1\", \"container\": \"reloader\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.05521506266909612}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-nats-2\", \"container\": \"nats\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.05521506266909612}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-nats-2\", \"container\": \"reloader\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.05521506266909612}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-ui-7f868dfff4-mh2c9\", \"container\": \"testkube-cloud-ui\", \"cpu_value\": 50.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.05521506266909612}, {\"namespace\": \"testkube\", \"pod\": \"testkube-enterprise-worker-service-78c654bddb-crjgn\", \"container\": \"testkube-worker-service\", \"cpu_value\": 75.0, \"cpu_units\": \"m\", \"milli_cpus\": 0.0828225940036442}]}}, {\"mode\": \"vega-lite\"});\n",
       "</script>"
      ],
      "text/plain": [
       "alt.Chart(...)"
      ]
     },
     "execution_count": 7,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "alt.Chart(\n",
    "    final_df.select( # type: ignore\n",
    "        pl.col(\"*\").exclude(\"milli_cpus\"),\n",
    "        milli_cpus=(pl.col(\"milli_cpus\") / total_milli_cpus) * 100\n",
    "    )\n",
    ").mark_bar().encode(\n",
    "    x='namespace',\n",
    "    y='sum(milli_cpus)',\n",
    "    color='container',\n",
    ")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.13.1"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}
